Make sure to have sqlalchemy-access installed for uploading back into the database

Only needs to be done once! 

You can't be connected to VPN due to encryption issues. Install using Anaconda prompt when disconnected from VPN

In [1]:
# only needed if you want to reupload to Microsoft Access database
#pip install sqlalchemy-access

tblcd_FluidLevelStatusCode

# Import Libraries

In [1]:
import pandas as pd #for creating dataframe
import pyodbc #working with ODBC databases
import numpy as np # for locating values in dataframes
from datetime import datetime # for obtaining today's date
import os #for working with directories

In [2]:
#optional- set dfs to show actual numbers rather than scientific notation
pd.set_option('display.float_format', lambda x: '%.5f' % x) 

# Manage directories

In [3]:
# my default directory is c:\Users\bmilinic\OneDrive - DOI\Documents\Python\bemidji
defaultdirectory = os.getcwd()
print(defaultdirectory)

c:\Users\bmilinic\OneDrive - DOI\Documents\Python\bemidji


In [4]:
# connect to the shared drive which holds the databases and files (must be connected to VPN)
os.chdir('P:/0083/analysis/DataCompilation/DataCompilationPy/create_site_info_files')

# Import data

In [5]:
# from GWSI Python output 
dfbmj3 = pd.read_csv(r'data_inputs/gwsi_current/bmj3_fromPy.csv')
dfrmk = pd.read_csv(r'data_inputs/gwsi_current/bmj_rmk_fromPy.csv')
# from Python outout aquarius and MLR 
dfaq = pd.read_csv(r'data_inputs/aquarius/Referencepoints_updatedMP_fromPy.csv') 
dfmlr = pd.read_csv(r'data_inputs/MLR/MLR_fromPy.csv')

In [6]:
# Import data from Microsoft Access Using PYODBC
##Gfe_db = pyodbc.connect(r'Driver={Microsoft Access Driver (*.mdb, *.accdb)};DBQ=P:\0083\analysis\DataCompilation\DataCompilationPy\local_access_db\BemidjiMasterSiteData_fe.accdb;')
Gfe_db = pyodbc.connect(r'Driver={Microsoft Access Driver (*.mdb, *.accdb)};DBQ=P:\0083\analysis\DataCompilation\BemidjiMasterSiteData_fe.accdb;')
c_fe = Gfe_db.cursor()

# backend db
#Gbe_db = pyodbc.connect(r'Driver={Microsoft Access Driver (*.mdb, *.accdb)};DBQ=P:\0083\analysis\DataCompilation\BackendData\BemidjiMasterSiteData_be.accdb;')
#c_be = Gfe_db.cursor()


In [7]:
# tables
tblSites = pd.read_sql('select * from tblSites', Gfe_db)

C:\Users\bmilinic\AppData\Local\Temp\1\ipykernel_29024\1723478965.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  tblSites = pd.read_sql('select * from tblSites', Gfe_db)


In [8]:
tblWells = pd.read_sql('select * from tblWells', Gfe_db)

C:\Users\bmilinic\AppData\Local\Temp\1\ipykernel_29024\3479789449.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  tblWells = pd.read_sql('select * from tblWells', Gfe_db)


In [9]:
tblCores = pd.read_sql('select * from tblCores', Gfe_db)

C:\Users\bmilinic\AppData\Local\Temp\1\ipykernel_29024\401927510.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  tblCores = pd.read_sql('select * from tblCores', Gfe_db)


In [10]:
tblOE = pd.read_sql('select * from tblOtherEquipment', Gfe_db)

C:\Users\bmilinic\AppData\Local\Temp\1\ipykernel_29024\3101191155.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  tblOE = pd.read_sql('select * from tblOtherEquipment', Gfe_db)


In [11]:
# cd tables
tblcd_LocalUseCode = pd.read_sql("select * from tblcd_LocalUseCode", Gfe_db)  

C:\Users\bmilinic\AppData\Local\Temp\1\ipykernel_29024\4250914646.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  tblcd_LocalUseCode = pd.read_sql("select * from tblcd_LocalUseCode", Gfe_db)


In [12]:
tblcd_CasingMaterial = pd.read_sql("select * from tblcd_CasingMaterial", Gfe_db)  

C:\Users\bmilinic\AppData\Local\Temp\1\ipykernel_29024\1707575347.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  tblcd_CasingMaterial = pd.read_sql("select * from tblcd_CasingMaterial", Gfe_db)


In [13]:
tblcd_ScreenMaterial = pd.read_sql("select * from tblcd_ScreenMaterial", Gfe_db)

C:\Users\bmilinic\AppData\Local\Temp\1\ipykernel_29024\950181360.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  tblcd_ScreenMaterial = pd.read_sql("select * from tblcd_ScreenMaterial", Gfe_db)


In [14]:
tblcd_OpeningType = pd.read_sql("select * from tblcd_OpeningType", Gfe_db)  

C:\Users\bmilinic\AppData\Local\Temp\1\ipykernel_29024\4104256497.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  tblcd_OpeningType = pd.read_sql("select * from tblcd_OpeningType", Gfe_db)


In [15]:
''' IF LIKE SCRIPT 03, DOING IT ALL AT ONCE SLOWS IT DOWN
# from GWSI Python output 
dfbmj3 = pd.read_csv(r'data_inputs/gwsi_old/bmj3_fromPy.csv')
dfrmk = pd.read_csv(r'data_inputs/gwsi_old/bmj_rmk_fromPy.csv')
# from Python outout aquarius and MLR 
dfaq = pd.read_csv(r'data_inputs/aquarius/Referencepoints_updatedMP_fromPy.csv') 
dfmlr = pd.read_csv(r'data_inputs/MLR/MLR_fromPy.csv')

# Import data from Microsoft Access Using PYODBC
Gfe_db = pyodbc.connect(r'Driver={Microsoft Access Driver (*.mdb, *.accdb)};DBQ=P:\0083\analysis\DataCompilation\DataCompilationPy\local_access_db\BemidjiMasterSiteData_fe.accdb;')
c_fe = Gfe_db.cursor()
# tables
tblSites = pd.read_sql('select * from tblSites', Gfe_db)
tblWells = pd.read_sql('select * from tblWells', Gfe_db)
tblCores = pd.read_sql('select * from tblCores', Gfe_db)
tblOE = pd.read_sql('select * from tblOtherEquipment', Gfe_db)
# cd tables
tblcd_LocalUseCode = pd.read_sql("select * from tblcd_LocalUseCode", Gfe_db)  
tblcd_CasingMaterial = pd.read_sql("select * from tblcd_CasingMaterial", Gfe_db)   
tblcd_ScreenMaterial = pd.read_sql("select * from tblcd_ScreenMaterial", Gfe_db)   
tblcd_OpeningType = pd.read_sql("select * from tblcd_OpeningType", Gfe_db)  
'''
print('ignore')

ignore


<>:1: SyntaxWarning: invalid escape sequence '\D'
<>:1: SyntaxWarning: invalid escape sequence '\D'
C:\Users\bmilinic\AppData\Local\Temp\1\ipykernel_29024\746478217.py:1: SyntaxWarning: invalid escape sequence '\D'
  ''' IF LIKE SCRIPT 03, DOING IT ALL AT ONCE SLOWS IT DOWN


In [16]:
# close the cursors and connections
c_fe.close()
Gfe_db.close()

## Merge tbleSites (local access) with GWSI (retrieved) to update the USGS station names

In [17]:
dfbmj3.GWSI_USGS_siteno

0      473429095051006
1      473424095052912
2      473424095052906
3      473423095052902
4      473412095050801
            ...       
613    473419095052503
614    473419095052304
615    473425095051601
616    473423095051501
617    473424095053001
Name: GWSI_USGS_siteno, Length: 618, dtype: int64

In [18]:
dfbmj3[dfbmj3['GWSI_USGS_siteno'] == 473430095051401]

,GWSI_AgencyCode,GWSI_USGS_siteno,GWSI_GWSISiteType,GWSI_GWSIUseOfSite,GWSI_GWSIAquiferType,GWSI_GWSIPrimaryAquifer,GWSI_GWSINationalAquifer,GWSI_TotalWellDepth_ftBLS,GWSI_WellCasingInnerDiameter_inches,GWSI_CasingMaterial,...,GWSI_MP_SequenceNo,GWSI_CONS_RecordType,GWSI_MP_LastUpdate,GWSI_MP_WebReady,GWSI_MP_height_m,GWSI_MP_Altitude_m,GWSI_CONS_SequenceNo,GWSI_HOLE_SequenceNo,GWSI_CSNG_SequenceNo,GWSI_OPEN_SequenceNo
205,USGS,473430095051401,GW,O,U,112GSSG,N100GLCIAL,27.53000,0.75000,Y,...,NaN,NaN,NaN,NaN,NaN,NaN,1.00000,1.00000,1.00000,1.00000
206,USGS,473430095051401,GW,O,U,112GSSG,N100GLCIAL,27.53000,0.75000,V,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.00000,NaN


In [19]:
dfrmk[dfrmk['GWSI_USGS_siteno'] == 473430095051401]

,GWSI_AgencyCode,GWSI_USGS_siteno,GWSI_GWSI_RMK,GWSI_GWSI_RMK_Date,GWSI_GWSI_RMK_SequenceNo
208,USGS,473430095051401,NaN,NaN,NaN


In [20]:
display(tblSites[tblSites['USGS_siteno'] == '473423095051901'])
display(tblSites[tblSites['USGS_siteno'] == 473423095051901])

,AgencyCode,Wellcon_SiteRecordID,DatabasePointType,USGS_siteno,ProjectAccountNumber,USGS_StationName,LocalSiteName,StateDistrictCode,CountryCode,StateCode,...,SiteEstablishedForWhom,OilSmell,SiteVerticalSource,SiteActiveStatus,XLocal_m_FromArc,YLocal_m_FromArc,OnNWT,Comments_Status,ApproxRemovalDate,StudySite
788,USGS,NaN,Data,473423095051901,MN-09500,2209 Bemidji Toxics Research Site 000078...,2209,MN,US,27.00000,...,Jared Trost,Y,D-22.2,Active,NaN,NaN,NaN,None,None,Bemidji


,AgencyCode,Wellcon_SiteRecordID,DatabasePointType,USGS_siteno,ProjectAccountNumber,USGS_StationName,LocalSiteName,StateDistrictCode,CountryCode,StateCode,...,SiteEstablishedForWhom,OilSmell,SiteVerticalSource,SiteActiveStatus,XLocal_m_FromArc,YLocal_m_FromArc,OnNWT,Comments_Status,ApproxRemovalDate,StudySite


In [21]:
deletedis = dfbmj3[['GWSI_USGS_siteno', 'GWSI_BottomOfScreenDepth_ftBLS', 'GWSI_TopOfScreenDepth_ftBLS']].copy()
deletedis[deletedis['GWSI_USGS_siteno'] == 473430095051401]

,GWSI_USGS_siteno,GWSI_BottomOfScreenDepth_ftBLS,GWSI_TopOfScreenDepth_ftBLS
205,473430095051401,27.37000,26.60000
206,473430095051401,NaN,NaN


In [22]:
tblSites['Loc/MiscCom']

0                                                    None
1                                                    None
2                                                    None
3                                                    None
4                                                    None
                              ...                        
1796                            Core taken near well 9015
1797    Installed this vapor well next to WT1712 (wate...
1798    Installed this vapor well next to 1711G (water...
1799               Core taken near well 956 in spray zone
1800               Core taken near well 956 in spray zone
Name: Loc/MiscCom, Length: 1801, dtype: object

In [23]:
# merge tblSites and dfbmj3: match up datatypes of the columns that the dfs will be merged on
tblSites2 = tblSites
tblSites2['USGS_siteno'] = tblSites2['USGS_siteno'].fillna(0).astype('int64')
dfcomplete = pd.merge(tblSites2.add_prefix('SITE_'), dfbmj3, left_on='SITE_USGS_siteno', right_on='GWSI_USGS_siteno', how='left')
# turn zeros back into NaN
dfcomplete['SITE_USGS_siteno'].replace(0, np.nan, inplace=True) 
print(dfcomplete.shape)
# Merge tbl wells
dfcomplete = pd.merge(dfcomplete, tblWells.add_prefix('WELL_'), left_on='SITE_LocalSiteName', right_on='WELL_LocalSiteName', how='left')
print(dfcomplete.shape)
# Merge tblOE
dfcomplete = pd.merge(dfcomplete, tblOE.add_prefix('OE_'), left_on='SITE_LocalSiteName', right_on='OE_LocalSiteName', how='left')
print(dfcomplete.shape)
# Merge tblCores
dfcomplete = pd.merge(dfcomplete, tblCores.add_prefix('CORE_'), left_on='SITE_LocalSiteName', right_on='CORE_LocalSiteName', how='left')
print(dfcomplete.shape)
# Merge dfmlr
dfcomplete = pd.merge(dfcomplete, dfmlr.add_prefix('MLR_'), left_on='SITE_USGS_siteno', right_on='MLR_site_no', how='left')
print(dfcomplete.shape)
# Merge dfaq
dfcomplete = pd.merge(dfcomplete, dfaq.add_prefix('AQ_'), left_on='SITE_USGS_siteno', right_on='AQ_site', how='left')
print(dfcomplete.shape)

(1806, 123)
(1808, 217)
(1818, 250)
(2585, 292)
(2585, 334)
(2585, 350)


C:\Users\bmilinic\AppData\Local\Temp\1\ipykernel_29024\1370290455.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  dfcomplete['SITE_USGS_siteno'].replace(0, np.nan, inplace=True)


In [24]:
# delete this
deletedis = dfcomplete[['SITE_LocalSiteName','SITE_LandSurfaceAltitude_mASL_NAVD88','GWSI_LandSurfaceAltitude_mASL_NAVD88', 'MLR_alt_va']]
deletedis[deletedis['SITE_LocalSiteName']=='2106']

,SITE_LocalSiteName,SITE_LandSurfaceAltitude_mASL_NAVD88,GWSI_LandSurfaceAltitude_mASL_NAVD88,MLR_alt_va
1013,2106,433.36400,NaN,NaN
1014,2106,433.36400,NaN,NaN
1015,2106,433.36400,NaN,NaN
1016,2106,433.36400,NaN,NaN
1017,2106,433.36400,NaN,NaN
1018,2106,433.36400,NaN,NaN
1019,2106,433.36400,NaN,NaN
1020,2106,433.36400,NaN,NaN
1021,2106,433.36400,NaN,NaN
1022,2106,433.36400,NaN,NaN


# Conversion

In [25]:
#V2
# Create function to convert units
def convert_feet_to_meters(columns, df):
    for column in columns:
        if 'ft' in column:
            new_column_name = column.replace('ft','m')
            #df = df.rename(columns={column: new_column_name})
            df[new_column_name] = df[column].mul(.3048).round(3)
        else: 
            df[column] = df[column].mul(.3048).round(3)
    return df

def convert_inches_to_cm(columns, df, decimal_places=1): # default value for ".round()" becomes 1 if you dont put anything there
    for column in columns:
        if 'inches' in column:
            new_column_name = column.replace('inches','cm')
            #df = df.rename(columns={column: new_column_name})
            df[new_column_name] = df[column].mul(2.54).round(decimal_places)
        else: 
            df[column] = df[column].mul(2.54).round(decimal_places)
    return df

In [26]:
# create list of which variables you want to be converted from ft to m and convert
columns_to_convert = ['GWSI_LandSurfaceAltitude_ftASL_NAVD88',
                      'GWSI_TopOfScreenDepth_ftBLS',
                      'GWSI_BottomOfScreenDepth_ftBLS',
                      'GWSI_TotalWellDepth_ftBLS',
                      'GWSI_MP_height_ft',
                      "SITE_TotalBoring/DrillingDepth_ftBLS",
                      'MLR_alt_va',
                      'MLR_hole_depth_va',
                      "OE_OtherEquipStickupLength_ft",
                      'CORE_CoreStickupLength_ft',
                      'CORE_CoringBegan_ftBLS',
                      'CORE_CoringEnded_ftBLS',
                      'CORE_CoreRecoveryLength_ft',
                      'CORE_TotalCoreLengthPounded_ft',
                      'MLR_well_depth_va',
                      'AQ_Elevation']
dfcomplete = convert_feet_to_meters(columns_to_convert, dfcomplete)

# repeat for inches to cm. Note the argument for rounding
columns_to_convert = ['GWSI_DiameterOfDrillHole_inches',
                      'GWSI_WellCasingInnerDiameter_inches']
dfcomplete = convert_inches_to_cm(columns_to_convert, dfcomplete, 1)  # rounds to one decimal
columns_to_convert = ['GWSI_WidthOfOpeningsInOpenInterval_inches']
dfcomplete = convert_inches_to_cm(columns_to_convert, dfcomplete, 3) # round to 3 decimals


#### AQ ELEVATION

In [27]:
# CHANGE DATE TIME FORMATS

# convert datetime format to string
dfcomplete['SITE_DateOfConstruction'] = dfcomplete['SITE_DateOfConstruction'].dt.strftime('%Y%m%d')
# replace year 0001 values with 1700 since datetime plays nicer
dfcomplete['AQ_ValidFrom'] = dfcomplete['AQ_ValidFrom'].replace('0001-01-01T00:00:00.0000000+00:00', '1700-01-01T00:00:00.0000000+00:00')
# convert the string into a datetime object
dfcomplete['AQ_ValidFrom'] = pd.to_datetime(dfcomplete['AQ_ValidFrom'], utc=True, format= '%Y-%m-%dT%H:%M:%S.%f%z')
# convert the datatime values to strings with new format (YYYYMMDD)
dfcomplete['AQ_ValidFrom'] = dfcomplete['AQ_ValidFrom'].dt.strftime('%Y%m%d')
# repeat for MP_EndDate
dfcomplete['AQ_DecommissionedDate'] = pd.to_datetime(dfcomplete['AQ_DecommissionedDate'], utc=True, format= '%Y-%m-%dT%H:%M:%S.%f%z')
dfcomplete['AQ_DecommissionedDate'] = dfcomplete['AQ_DecommissionedDate'].dt.strftime('%Y%m%d')


In [28]:
# UPDATE RECORDS BY COMBINING COLUMNS (make sure you're using the authoritative source!)
# .where() is choosing between (x.notna(), y) so it chooses x ONLY where it is not nan and if it is nan...it chooses y
# station name
dfcomplete['USGS_StationName'] = dfcomplete['GWSI_USGS_StationName'].where(dfcomplete['GWSI_USGS_StationName'].notna(), dfcomplete['SITE_USGS_StationName']) 
dfcomplete['USGS_StationName'] = dfcomplete['MLR_station_nm'].where(dfcomplete['MLR_station_nm'].notna(), dfcomplete['USGS_StationName'])
# date of construction - make sure date time format was updated (block of code above)
dfcomplete['DateOfConstruction'] = dfcomplete['GWSI_DateOfConstruction'].where(dfcomplete['GWSI_DateOfConstruction'].notna(), dfcomplete['SITE_DateOfConstruction'])
dfcomplete['DateOfConstruction'] = dfcomplete['MLR_construction_dt'].where(dfcomplete['MLR_construction_dt'].notna(), dfcomplete['DateOfConstruction'])
# Landsurfaceelvation
dfcomplete['LandSurfaceAltitude_mASL_NAVD88'] = dfcomplete['SITE_LandSurfaceAltitude_mASL_NAVD88'].where(dfcomplete['SITE_LandSurfaceAltitude_mASL_NAVD88'].notna(), dfcomplete['GWSI_LandSurfaceAltitude_mASL_NAVD88'])
dfcomplete['LandSurfaceAltitude_mASL_NAVD88'] = dfcomplete['MLR_alt_va'].where(dfcomplete['MLR_alt_va'].notna(), dfcomplete['LandSurfaceAltitude_mASL_NAVD88'])
# drillingdepth
dfcomplete['TotalBoring/DrillingDepth_mBLS'] = dfcomplete['MLR_hole_depth_va'].where(dfcomplete['MLR_hole_depth_va'].notna(), dfcomplete["SITE_TotalBoring/DrillingDepth_mBLS"])
# well depth
dfcomplete['TotalWellDepth_mBLS'] = dfcomplete['MLR_well_depth_va'].where(dfcomplete['MLR_well_depth_va'].notna(), dfcomplete['GWSI_TotalWellDepth_ftBLS'])
# MP height
dfcomplete['well_MP_height_m'] = dfcomplete['AQ_Elevation'].where(dfcomplete['AQ_Elevation'].notna(), dfcomplete['GWSI_MP_height_m'])
# MP begin date
dfcomplete['MP_BeginDate'] = dfcomplete['AQ_ValidFrom'].where(dfcomplete['AQ_ValidFrom'].notna(), dfcomplete['GWSI_MP_BeginDate'])
# MP end date
dfcomplete['MP_EndDate'] = dfcomplete['AQ_DecommissionedDate'].where(dfcomplete['AQ_DecommissionedDate'].notna(), dfcomplete['GWSI_MP_EndDate'])


In [29]:
dfcomplete['GWSI_BottomOfScreenDepth_mBLS'] 

0      NaN
1      NaN
2      NaN
3      NaN
4      NaN
        ..
2580   NaN
2581   NaN
2582   NaN
2583   NaN
2584   NaN
Name: GWSI_BottomOfScreenDepth_mBLS, Length: 2585, dtype: float64

In [30]:
# calculate new columns
dfcomplete['MeasuringPointElevation_mASL_NAVD88'] = (dfcomplete['LandSurfaceAltitude_mASL_NAVD88'] + dfcomplete['well_MP_height_m']).round(3) #renamed for Wells
dfcomplete['TopOfScreenElevation_mASL_NAVD88'] = (dfcomplete['LandSurfaceAltitude_mASL_NAVD88'] - dfcomplete['GWSI_TopOfScreenDepth_mBLS']).round(3)
dfcomplete['BottomOfScreenElevation_mASL_NAVD88'] = (dfcomplete['LandSurfaceAltitude_mASL_NAVD88'] - dfcomplete['GWSI_BottomOfScreenDepth_mBLS']).round(3)
dfcomplete['MidOfScreenElevation_mASL_NAVD88'] = (dfcomplete['TopOfScreenElevation_mASL_NAVD88'] + dfcomplete['BottomOfScreenElevation_mASL_NAVD88']).round(3)
dfcomplete['MidOfScreenElevation_mASL_NAVD88'] = dfcomplete['MidOfScreenElevation_mASL_NAVD88'].div(2).round(3)
dfcomplete['ScreenLength_m'] = (dfcomplete['GWSI_BottomOfScreenDepth_mBLS'] - dfcomplete['GWSI_TopOfScreenDepth_mBLS']).round(3)#renamed for Wells

# create indicator columns in tblSites for which sites exist in each of site type tables (wells, cores, other equipment)
dfcomplete['WellSite'] = np.where(dfcomplete['SITE_LocalSiteName'].isin(tblWells['LocalSiteName']), 1, 0) # inserts 1 if true and 0 if false
dfcomplete['CoreSite'] = np.where(dfcomplete['SITE_LocalSiteName'].isin(tblCores['LocalSiteName']), 1, 0)
dfcomplete['OtherEquipmentSite'] = np.where(dfcomplete['SITE_LocalSiteName'].isin(tblOE['LocalSiteName']), 1, 0)

In [31]:
dfcomplete[['MeasuringPointElevation_mASL_NAVD88', 'TopOfScreenElevation_mASL_NAVD88', 'MidOfScreenElevation_mASL_NAVD88', 'ScreenLength_m']].sample(10)

,MeasuringPointElevation_mASL_NAVD88,TopOfScreenElevation_mASL_NAVD88,MidOfScreenElevation_mASL_NAVD88,ScreenLength_m
1315,NaN,NaN,NaN,NaN
1153,NaN,NaN,NaN,NaN
524,NaN,NaN,NaN,NaN
1745,433.63600,NaN,NaN,NaN
2574,NaN,NaN,NaN,NaN
36,NaN,NaN,NaN,NaN
671,NaN,NaN,NaN,NaN
1976,NaN,NaN,NaN,NaN
133,NaN,NaN,NaN,NaN
1826,NaN,NaN,NaN,NaN


In [32]:
#Rename columns
dfcomplete = dfcomplete.rename(columns={'SITE_OnNWT':'OnNorthWellTransect',
                                        'SITE_NWTPosition_m':'DistanceFromCenterOfNorthOilBody_m',
                                        'LandSurfaceAltitude_mASL_NAVD88':'LandSurfaceElevation_mASL_NAVD88',
                                        'SITE_Loc/MiscCom':'Comments_Miscellaneous',
                                        "SITE_LocalSiteName":"LocalSiteName",
                                        "SITE_StudySite":"StudySite",
                                        "SITE_AgencyCode":"AgencyCode",
                                        "SITE_USGS_siteno":"USGS_siteno",
                                        "SITE_XcoordUTMNAD83_m":"XcoordUTMNAD83_m",
                                        "SITE_YcoordUTMNAD83_m":"YcoordUTMNAD83_m",
                                        "SITE_Comments_DateOfConstruction":"Comments_DateOfConstruction",
                                        "SITE_SiteActiveStatus":"SiteActiveStatus",
                                        "SITE_Comments_Status":"Comments_Status",
                                        "SITE_ApproxRemovalDate":"ApproxRemovalDate",
                                        "OE_OtherEquipStickupLength_m":'OtherEquip_MP_height_m', #For Other equipment data release
                                        'SITE_NameOfContractor':'NameOfContractor',
                                        'SITE_DrillerName':'DrillerName',
                                        'OE_OtherEquipmentPurpose':'OtherEquipmentPurpose',
                                        'OE_Comments_Equipment':'Comments_Equipment',
                                        'OE_Comments':'Comments',
                                        'CORE_CoreStickupLength_m':'Core_MP_height_m',
                                        'CORE_TotalCoreLengthPounded_m':'CoreLengthPounded_m',
                                        'CORE_LocalSiteCoreSubName':'LocalSiteCoreSubName',
                                        'CORE_CoringBegan_mBLS':'CoringBegan_mBLS', 
                                        'CORE_CoringEnded_mBLS':'CoringEnded_mBLS', 
                                        'CORE_CoreRecoveryLength_m':'CoreRecoveryLength_m',
                                        'CORE_CoreBarrelType':'CoreBarrelType', 
                                        'CORE_SubsurfaceCoreZone':'SubsurfaceCoreZone', 
                                        'CORE_FreeProductOilPresentInCore':'FreeProductOilPresentInCore', 
                                        'CORE_DrillersFieldComments':'DrillersFieldComments',
                                        'GWSI_DiameterOfDrillHole_cm':'DiameterOfDrillHole_cm',
                                        'GWSI_WellCasingInnerDiameter_cm':'WellCasingInnerDiameter_cm',
                                        'GWSI_WidthOfOpeningsInOpenInterval_cm':'WidthOfOpeningsInOpenInterval_cm'
                                        }) 
                                        

## NaN conversion

In [33]:
dfcomplete['USGS_siteno'] =  dfcomplete['USGS_siteno'].astype('Int64')
dfcomplete['USGS_siteno'] = 'x' + dfcomplete['USGS_siteno'].astype(str)
dfcomplete['USGS_siteno'] = dfcomplete['USGS_siteno'].replace('x<NA>', np.nan)

In [34]:
dfcomplete

,AgencyCode,SITE_Wellcon_SiteRecordID,SITE_DatabasePointType,USGS_siteno,SITE_ProjectAccountNumber,SITE_USGS_StationName,LocalSiteName,SITE_StateDistrictCode,SITE_CountryCode,SITE_StateCode,...,MP_BeginDate,MP_EndDate,MeasuringPointElevation_mASL_NAVD88,TopOfScreenElevation_mASL_NAVD88,BottomOfScreenElevation_mASL_NAVD88,MidOfScreenElevation_mASL_NAVD88,ScreenLength_m,WellSite,CoreSite,OtherEquipmentSite
0,USGS,NaN,None,NaN,MN-09500,None,15-N6,MN,US,27.00000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1,0
1,USGS,NaN,None,NaN,MN-09500,None,15-N7,MN,US,27.00000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1,0
2,USGS,NaN,None,NaN,MN-09500,None,15-N8,MN,US,27.00000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1,0
3,USGS,NaN,None,NaN,MN-09500,None,15-N9,MN,US,27.00000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1,0
4,USGS,NaN,None,NaN,MN-09500,None,15-N10,MN,US,27.00000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2580,USGS,NaN,Data,NaN,MN-09500,None,1706,MN,US,27.00000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1,0
2581,USGS,NaN,Data,NaN,MN-09500,None,1706,MN,US,27.00000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1,0
2582,USGS,NaN,Data,NaN,MN-09500,None,1706,MN,US,27.00000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1,0
2583,USGS,NaN,Data,NaN,MN-09500,None,1706,MN,US,27.00000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1,0


In [35]:
chk = dfcomplete[dfcomplete['LocalSiteName'].str.startswith('21', na=False)]
chk[['LocalSiteName', 'ScreenLength_m']]

,LocalSiteName,ScreenLength_m
988,2101,0.23400
989,2101,NaN
990,2103A,0.75000
991,2103B,0.23500
992,2103B,NaN
...,...,...
1058,2112,NaN
1059,2112,NaN
1060,2112,NaN
1061,2112,NaN


# TABLES

## MasterSiteList

In [36]:
#create mastersitelist data release
mastersitelist = dfcomplete.loc[:, ["LocalSiteName",
                                    "StudySite",
                                    "AgencyCode",
                                    "USGS_siteno",
                                    "USGS_StationName",
                                    "XcoordUTMNAD83_m",
                                    "YcoordUTMNAD83_m",
                                    "LandSurfaceElevation_mASL_NAVD88",
                                    "OnNorthWellTransect",
                                    "DistanceFromCenterOfNorthOilBody_m",
                                    "WellSite",
                                    "CoreSite",
                                    "OtherEquipmentSite",
                                    "TopOfScreenElevation_mASL_NAVD88",
                                    "BottomOfScreenElevation_mASL_NAVD88",
                                    "MidOfScreenElevation_mASL_NAVD88",
                                    "DateOfConstruction",
                                    "Comments_DateOfConstruction",
                                    "SiteActiveStatus",
                                    "Comments_Status",
                                    "ApproxRemovalDate"]]

# format date
mastersitelist['LocalSiteName'] = 'x' + mastersitelist['LocalSiteName'].astype(str)

# make nan values uniform
null_cells = mastersitelist.isnull()
mastersitelist = mastersitelist.astype(str).mask(null_cells, np.nan)
# delete dublicates
mastersitelist2 = mastersitelist.drop_duplicates(subset='LocalSiteName')
mastersitelist2.shape

(1801, 21)

## other equipment

In [37]:
# create OE data release (this merge is unqiue and must be done last...'LocalUseCode' when merged, needs to use the "left_on" of which ever table (OE or Core etc). Otherwise you have to do some renaming magic to keep all the 'LocalUseCode' columns seperate and change last minute while deleting old columns toi avoid repeating the same col name)
tblOE2 = pd.merge(dfcomplete, tblcd_LocalUseCode[['LocalUseCode','Comments_UseCode']], left_on='OE_LocalUseCode', right_on='LocalUseCode',how='left') #if you leave out the "how" (like in the original) then # of rows plummits
tblOE2 = tblOE2.loc[:,["LocalSiteName",
                       "StudySite",
                       "LocalUseCode",
                       "Comments_UseCode",
                       "USGS_siteno",
                       "USGS_StationName",
                       "XcoordUTMNAD83_m",
                       "YcoordUTMNAD83_m",
                       "OnNorthWellTransect",
                       "DistanceFromCenterOfNorthOilBody_m",
                       "LandSurfaceElevation_mASL_NAVD88",
                       "OtherEquip_MP_height_m",
                       "DateOfConstruction",
                       "Comments_DateOfConstruction",
                       "NameOfContractor",
                       "DrillerName",
                       "OtherEquipmentPurpose",
                       "Comments_Equipment",
                       "Comments",
                       "Comments_Miscellaneous"]]
# delete NAs and dublicates
#tblOE2 = tblOE2.dropna(axis=0, subset=['LocalUseCode']) # OLD filter that drops all rows where LocalUseCode is na
tblOE2 = tblOE2[tblOE2['LocalUseCode'].notnull() | tblOE2['Comments'].notnull()] # NEW filter
tblOE2 = tblOE2.drop_duplicates(subset='LocalSiteName')

# final touches
tblOE2['LocalSiteName'] = 'x' + tblOE2['LocalSiteName'].astype(str)
# make nan values uniform
null_cells = tblOE2.isnull()
tblOE2 = tblOE2.astype(str).mask(null_cells, np.nan)

## Cores

In [38]:
# Create one extra column which needs to come after the column merges
dfcomplete['TopOfCoreElevation_mASL_NAVD88'] = dfcomplete['LandSurfaceElevation_mASL_NAVD88'] - dfcomplete['CoringBegan_mBLS']
dfcomplete['TopOfCoreElevation_mASL_NAVD88'] = dfcomplete['TopOfCoreElevation_mASL_NAVD88'].round(3)
# Migrate local use code 
tblCores2 = pd.merge(dfcomplete, tblcd_LocalUseCode[['LocalUseCode','Comments_UseCode']], left_on='CORE_LocalUseCode', right_on='LocalUseCode',how='left')
# create data release in column order
tblCores2 = tblCores2.loc[:, ["LocalSiteName",
                              "StudySite",
                              "LocalUseCode",
                              "Comments_UseCode",
                              "USGS_siteno",
                              "USGS_StationName",
                              "LocalSiteCoreSubName",
                              "XcoordUTMNAD83_m",
                              "YcoordUTMNAD83_m",
                              "OnNorthWellTransect",
                              "DistanceFromCenterOfNorthOilBody_m",
                              "LandSurfaceElevation_mASL_NAVD88",
                              "Core_MP_height_m",
                              "DateOfConstruction",
                              "Comments_DateOfConstruction",
                              "TotalBoring/DrillingDepth_mBLS",
                              "NameOfContractor",
                              "DrillerName",
                              "CoringBegan_mBLS",
                              "CoringEnded_mBLS",
                              "CoreRecoveryLength_m",
                              "TopOfCoreElevation_mASL_NAVD88",
                              "CoreLengthPounded_m",
                              "CoreBarrelType",
                              "SubsurfaceCoreZone",
                              "FreeProductOilPresentInCore",
                              "DrillersFieldComments",
                              "Comments_Miscellaneous"]] 
# delete uneeded cols
tblCores2 = tblCores2.dropna(axis=0, subset=['Comments_UseCode'])
#Format columns and male nan values uniform
tblCores2['LocalSiteName'] = 'x' + tblCores2['LocalSiteName'].astype(str)
null_cells = tblCores2.isnull()
tblCores2 = tblCores2.astype(str).mask(null_cells, np.nan)

In [39]:
tblCores2[['LocalSiteName', 'TopOfCoreElevation_mASL_NAVD88']].sample(10)

,LocalSiteName,TopOfCoreElevation_mASL_NAVD88
2137,xGP35,NaN
799,x1115,428.889
2046,xGP15,NaN
536,x0415,NaN
1893,x9813,NaN
1858,x9616,NaN
1035,x2108,428.54
357,xL310-4.5,432.353
1818,x9508,NaN
1310,x1308,427.692


In [40]:
deletedis = tblCores2[['LocalSiteName', 'LandSurfaceElevation_mASL_NAVD88']]
deletedis[deletedis['LocalSiteName']=='x2106']

,LocalSiteName,LandSurfaceElevation_mASL_NAVD88
1013,x2106,433.364
1014,x2106,433.364
1015,x2106,433.364
1016,x2106,433.364
1017,x2106,433.364
1018,x2106,433.364
1019,x2106,433.364
1020,x2106,433.364
1021,x2106,433.364
1022,x2106,433.364


## Wells

In [41]:
# add tblcd info
tblWells3 = pd.merge(dfcomplete, tblcd_OpeningType[["TypeOfOpenInterval", "comments_OpeningType"]], left_on='GWSI_TypeOfOpenInterval', right_on='TypeOfOpenInterval', how='left')
tblWells3 = pd.merge(tblWells3, tblcd_LocalUseCode[["LocalUseCode", "Comments_UseCode"]], left_on='WELL_LocalUseCode', right_on='LocalUseCode',how='left')
tblWells3 = pd.merge(tblWells3, tblcd_CasingMaterial[["CasingMaterial", "Comments_CasingMaterial"]], left_on='GWSI_CasingMaterial', right_on='CasingMaterial', how='left')
tblWells3 = pd.merge(tblWells3, tblcd_ScreenMaterial[["ScreenMaterial", "Comments_ScreenMaterial"]], left_on='GWSI_ScreenMaterialType', right_on='ScreenMaterial', how='left')

# rename tblcd info
tblWells3 = tblWells3.rename(columns={"comments_OpeningType":'OpeningTypeDescription'})
tblWells3 = tblWells3.rename(columns={"Comments_CasingMaterial":'CasingMaterialDescription'})
tblWells3 = tblWells3.rename(columns={"Comments_ScreenMaterial":'ScreenMaterialDescription'})

In [42]:
#data release
# select a subset of data
tblWells3 = tblWells3.loc[:,["LocalSiteName",
                             "StudySite",
                             "LocalUseCode",
                             "Comments_UseCode",
                             "USGS_siteno",
                             "USGS_StationName",
                             "XcoordUTMNAD83_m",
                             "YcoordUTMNAD83_m",
                             "OnNorthWellTransect",
                             "DistanceFromCenterOfNorthOilBody_m",
                             "MeasuringPointElevation_mASL_NAVD88", 
                             "well_MP_height_m",
                             "LandSurfaceElevation_mASL_NAVD88",
                             "TopOfScreenElevation_mASL_NAVD88",
                             "BottomOfScreenElevation_mASL_NAVD88",
                             "ScreenLength_m",
                             "MidOfScreenElevation_mASL_NAVD88",
                             "TotalWellDepth_mBLS",
                             "TotalBoring/DrillingDepth_mBLS",
                             "DiameterOfDrillHole_cm",
                             "WellCasingInnerDiameter_cm",
                             "OpeningTypeDescription",
                             "WidthOfOpeningsInOpenInterval_cm",
                             "CasingMaterialDescription",
                             "ScreenMaterialDescription",
                             "DateOfConstruction",
                             "Comments_DateOfConstruction",
                             "NameOfContractor",
                             "DrillerName",
                             "Comments_Miscellaneous",
                             "SiteActiveStatus",
                             "Comments_Status",
                             "ApproxRemovalDate"]]  
# delete uneeded cols
tblWells3 = tblWells3.dropna(axis=0, subset=['Comments_UseCode'])
tblWells3 = tblWells3.drop_duplicates()#subset='LocalSiteName')
# formatting columns
tblWells3['LocalSiteName'] = 'x' + tblWells3['LocalSiteName'].astype(str)
#Make nan values uniform
null_cells = tblWells3.isnull()
tblWells3 = tblWells3.astype(str).mask(null_cells, np.nan)  


In [43]:
# Special exception: Casing material changes from Steel-Galvanized to Stainless Steel 
# leading to 2 casing sequence numbers and almost duplicate rows 

## track number of rows before filter/mask
row_count = tblWells3.shape[0]

## create masks for specific conditions
mask_duplicates = tblWells3.duplicated(subset='LocalSiteName', keep=False)
mask_contains_stainless = tblWells3['CasingMaterialDescription'].str.contains('Stainless Steel', na=False)
mask_contains_steel_galv = tblWells3['CasingMaterialDescription'].str.contains('Steel-Galvanized', na=False)

## Identify duplicate rows that contain "Stainless Steel" 
rows_to_drop = mask_duplicates & mask_contains_stainless
## Remove these
tblWells3 = tblWells3[~rows_to_drop].copy()

## For the remaining duplicate rows, update "Steel-Galvanized" to "Stainless Steel"
rows_to_update = mask_duplicates & mask_contains_steel_galv
tblWells3.loc[rows_to_update, 'CasingMaterialDescription'] = 'Stainless Steel'

## Print out cols deleted
print('Number of rows removed:', row_count - tblWells3.shape[0])

Number of rows removed: 5


In [44]:
tblWells3

,LocalSiteName,StudySite,LocalUseCode,Comments_UseCode,USGS_siteno,USGS_StationName,XcoordUTMNAD83_m,YcoordUTMNAD83_m,OnNorthWellTransect,DistanceFromCenterOfNorthOilBody_m,...,CasingMaterialDescription,ScreenMaterialDescription,DateOfConstruction,Comments_DateOfConstruction,NameOfContractor,DrillerName,Comments_Miscellaneous,SiteActiveStatus,Comments_Status,ApproxRemovalDate
8,x1217E,Bemidji,ww,groundwater observation well,x473429095051006,1217E 147N35W02DCBB 06,343106.91,5271121.65,1.0,326.57,...,Stainless Steel,Stainless Steel,20160726.0,This well was installed by B. Mason several ye...,USGS MN Water Science Center,B. Mason,Between well 925 nest and well 501 nest. Insta...,Active,NaN,NaN
10,x1602G-01,Bemidji,wv,unsaturated zone vapor sampling well,x473424095052912,1602G-01 Bemidji Toxics Research Site,342693.48,5271007.85,1.0,-97.51,...,Stainless Steel,Stainless Steel,20160802.0,NaN,USGS MN Water Science Center,B. Mason; J.W. Lund,Installed this vapor well next to WT1601 (wate...,Active,NaN,NaN
11,x707G-01,Bemidji,wv,unsaturated zone vapor sampling well,x473424095052906,707G-01 Bemidji Toxics Research Site,342714.17,5271007.89,1.0,-78.58,...,Stainless Steel,Stainless Steel,20160809.0,NaN,USGS MN Water Science Center,B. Mason; J.W. Lund,Installed this vapor well next to current 707 ...,Active,NaN,NaN
27,xCL-MW3,Cass Lake,wo,subsurface oil- and groundwater observation well,NaN,NaN,376368.69,5248417.18,NaN,NaN,...,NaN,NaN,NaN,Installation date unknown,Enbridge Energy,unknown,NaN,Active,NaN,NaN
57,x17.lk.sc1,Bemidji,wmp,mini-piezometer (shallow temporary well near o...,NaN,NaN,343136.15,5271114.97,NaN,NaN,...,NaN,NaN,NaN,NaN,USGS MN Water Science Center,Not applicable,NaN,Removed,temporary sampling location,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2432,x1506G-01,Bemidji,wv,unsaturated zone vapor sampling well,x473419095052403,1506G-01 Bemidji Toxics Research Site,342809.38,5270868.76,NaN,NaN,...,Stainless Steel,Stainless Steel,20150812.0,Installed vapor well at this location several ...,USGS MN Water Science Center,B. Mason; E. Smith,"This site was originally named 1428G, but chan...",Active,NaN,NaN
2523,x1509G-01,Bemidji,wv,unsaturated zone vapor sampling well,x473419095052304,1509G-01 Bemidji Toxics Research Site,342817.53,5270850.06,NaN,NaN,...,Stainless Steel,Stainless Steel,20150813.0,NaN,USGS MN Water Science Center,B. Mason; E. Smith,"This site was originally named 1427G, but chan...",Active,NaN,NaN
2530,x1707,Bemidji,ww,groundwater observation well,x473425095051601,1707 Bemidji Toxics Research Site 000078...,342985.77,5271041.53,NaN,NaN,...,PVC Threaded,PVC Threaded,20170626.0,NaN,USGS MN Water Science Center,B. Mason; A. Berg,Well installed next to well MW1 & PZ1 nest,Active,NaN,NaN
2531,x1708,Bemidji,ww,groundwater observation well,x473423095051501,1708 Bemidji Toxics Research Site 000078...,343004.18,5270987.14,NaN,NaN,...,PVC Threaded,PVC Threaded,20170626.0,NaN,USGS MN Water Science Center,B. Mason; A. Berg,Well installed next to well MW2 & PZ2 nest,Active,NaN,NaN


In [45]:
tblWells3[tblWells3['LocalSiteName'].str.startswith('x21', na=False)]

,LocalSiteName,StudySite,LocalUseCode,Comments_UseCode,USGS_siteno,USGS_StationName,XcoordUTMNAD83_m,YcoordUTMNAD83_m,OnNorthWellTransect,DistanceFromCenterOfNorthOilBody_m,...,CasingMaterialDescription,ScreenMaterialDescription,DateOfConstruction,Comments_DateOfConstruction,NameOfContractor,DrillerName,Comments_Miscellaneous,SiteActiveStatus,Comments_Status,ApproxRemovalDate
988,x2101,Bemidji,ww,groundwater observation well,x473430095051401,2101 Bemidji Toxics Research Site,343039.9603,5271168.795,1.0,284.46,...,Stainless Steel,Stainless Steel,20210621.0,NaN,USGS MN Water Science Center,"A. Berg, Z. Hillman",Located near Unnamed Lake,Active,NaN,NaN
990,x2103A,Bemidji,ww,groundwater observation well,x473429095051402,2103A Bemidji Toxics Research Site,343067.1912,5271160.053,1.0,305.82,...,Steel-Galvanized,Stainless Steel,20210622.0,NaN,USGS MN Water Science Center,"A. Berg, Z. Hillman",Located near Unnamed Lake shoreline,Active,NaN,NaN
991,x2103B,Bemidji,ww,groundwater observation well,x473429095051403,2103B Bemidji Toxics Research Site,343067.7493,5271159.855,1.0,306.25,...,Stainless Steel,Stainless Steel,20210623.0,NaN,USGS MN Water Science Center,"A. Berg, Z. Hillman",Located near Unnamed Lake shoreline,Active,NaN,NaN
993,x2103C,Bemidji,ww,groundwater observation well,x473429095051404,2103C Bemidji Toxics Research Site,343066.5213,5271160.262,1.0,305.29,...,Stainless Steel,Stainless Steel,20210623.0,NaN,USGS MN Water Science Center,"A. Berg, Z. Hillman",Located near Unnamed Lake shoreline,Active,NaN,NaN
995,x2104A,Bemidji,ww,groundwater observation well,x473429095051405,2104A Bemidji Toxics Research Site,343044.3673,5271153.903,1.0,282.46,...,Steel-Galvanized,Stainless Steel,20210623.0,NaN,USGS MN Water Science Center,"A. Berg, Z. Hillman",Located on hillside near Unnamed Lake,Active,NaN,NaN
996,x2104B,Bemidji,ww,groundwater observation well,x473429095051406,2104B Bemidji Toxics Research Site,343044.1993,5271153.152,1.0,282.0,...,Stainless Steel,Stainless Steel,20210624.0,NaN,USGS MN Water Science Center,"A. Berg, Z. Hillman",Located on hillside near Unnamed Lake,Active,NaN,NaN
998,x2104C,Bemidji,ww,groundwater observation well,x473429095051407,2104C Bemidji Toxics Research Site,343044.5018,5271154.528,1.0,282.83,...,Stainless Steel,Stainless Steel,20210625.0,NaN,USGS MN Water Science Center,"A. Berg, Z. Hillman",Located on hillside near Unnamed Lake,Active,NaN,NaN
1000,x2102F,Bemidji,wf,failed well installation (or any installation),NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,20210622,"Attempted installation on 06/22/21, but well c...",USGS MN Water Science Center,"A. Berg, Z. Hillman",Located near 2105 well nest. This was a failed...,Removed,"failed well, no well installed",NaN
1001,x2105A,Bemidji,ww,groundwater observation well,x473430095051501,2105A Bemidji Toxics Research Site 000018...,343011.5996,5271150.569,1.0,251.15,...,PVC Threaded,PVC Threaded,20210815.0,NaN,USGS MN Water Science Center,"B. Mason, J. Letsos","Located on top of hill, down-gradient of 955 w...",Active,NaN,NaN
1002,x2105B,Bemidji,ww,groundwater observation well,x473430095051505,2105B Bemidji Toxics Research Site 000018...,343012.6681,5271151.244,1.0,252.4,...,PVC Threaded,PVC Threaded,20210815.0,NaN,USGS MN Water Science Center,"B. Mason, J. Letsos","Located on top of hill, down-gradient of 955 w...",Active,NaN,NaN


# bmj_rmk data release
organized seperately since bmj_rmk has repeating rows for every site number 

In [46]:
dfrmk2 = dfrmk.loc[:,['GWSI_AgencyCode', 'GWSI_USGS_siteno', 'GWSI_GWSI_RMK',
                      'GWSI_GWSI_RMK_Date', 'GWSI_GWSI_RMK_SequenceNo']]

# make columns the same datatype of interger
tblSites['USGS_siteno'] = tblSites['USGS_siteno'].fillna(0).astype('int64')
# dfrmk2's is already an interger
dfrmk2 = pd.merge(dfrmk2, tblSites[['USGS_siteno', 'LocalSiteName']], left_on= "GWSI_USGS_siteno", right_on= 'USGS_siteno', how='left')

# organize for data release
dfrmk3 = dfrmk2.loc[:, ['GWSI_AgencyCode',
                        'GWSI_USGS_siteno',
                        'LocalSiteName',
                        'GWSI_GWSI_RMK',
                        'GWSI_GWSI_RMK_Date',
                        'GWSI_GWSI_RMK_SequenceNo']]
# format numbers with 'x'
dfrmk3['GWSI_USGS_siteno'] = 'x' + dfrmk3['GWSI_USGS_siteno'].astype(str)
dfrmk3['LocalSiteName'] = 'x' + dfrmk3['LocalSiteName'].astype(str)

In [47]:
tblWells3[['LocalSiteName', 'TopOfScreenElevation_mASL_NAVD88', 'BottomOfScreenElevation_mASL_NAVD88', 'ScreenLength_m']]

,LocalSiteName,TopOfScreenElevation_mASL_NAVD88,BottomOfScreenElevation_mASL_NAVD88,ScreenLength_m
8,x1217E,420.645,420.557,0.088
10,x1602G-01,425.641,425.61,0.031
11,x707G-01,425.327,425.296,0.031
27,xCL-MW3,NaN,NaN,NaN
57,x17.lk.sc1,NaN,NaN,NaN
...,...,...,...,...
2432,x1506G-01,424.55,424.519,0.031
2523,x1509G-01,424.495,424.465,0.03
2530,x1707,420.005,419.764,0.241
2531,x1708,420.356,420.115,0.241


# Saving Every dataframe in a new directory

In [132]:
this is a syntax error to stop the script from running

SyntaxError: invalid syntax (620935946.py, line 1)

In [48]:
# create variable with today's date
date = datetime.today().strftime('%Y%m%d') 
# create variable for new folder/directory
dir = "data_outputs/"+date+"_datarelease" # new directory location and name
# create the directory unless it already exists...then skip
try:
    os.mkdir(dir)
except:
    pass
# save all the files there 
mastersitelist2.to_csv(dir+"/DataRelease_MasterSiteList.csv", index=False)
tblOE2.to_csv(dir+"/DataRelease_OtherEquipmentInformation.csv", index=False)
tblCores2.to_csv(dir+"/DataRelease_CoreInformation.csv", index=False)
tblWells3.to_csv(dir+"/DataRelease_WellConstructionInformation.csv", index=False)
dfrmk3.to_csv(dir+"/DataRelease_rmk3.csv", index=False)

In [49]:
print(f'saved to {os.getcwd()} --> {dir}')

saved to P:\0083\analysis\DataCompilation\DataCompilationPy\create_site_info_files --> data_outputs/20250221_datarelease


# Read in file to check

In [50]:
# directory
dir = r'P:\0083\analysis\DataCompilation\DataCompilationPy\create_site_info_files\data_outputs'
# today's date YYYYMMDD
date = '20250204'
# file 
file_path = f'{dir}\{date}_datarelease\DataRelease_WellConstructionInformation.csv'
# read into csv
chk = pd.read_csv(file_path, encoding='windows-1252')
chk[chk['LocalSiteName'].str.startswith('x21', na=False)]

<string>:6: SyntaxWarning: invalid escape sequence '\{'
<>:6: SyntaxWarning: invalid escape sequence '\{'
<>:6: SyntaxWarning: invalid escape sequence '\D'
<string>:6: SyntaxWarning: invalid escape sequence '\{'
<>:6: SyntaxWarning: invalid escape sequence '\{'
<>:6: SyntaxWarning: invalid escape sequence '\D'
C:\Users\bmilinic\AppData\Local\Temp\1\ipykernel_14040\2965541319.py:6: SyntaxWarning: invalid escape sequence '\{'
  file_path = f'{dir}\{date}_datarelease\DataRelease_WellConstructionInformation.csv'
C:\Users\bmilinic\AppData\Local\Temp\1\ipykernel_14040\2965541319.py:6: SyntaxWarning: invalid escape sequence '\D'
  file_path = f'{dir}\{date}_datarelease\DataRelease_WellConstructionInformation.csv'
<string>:6: SyntaxWarning: invalid escape sequence '\{'


,LocalSiteName,StudySite,LocalUseCode,Comments_UseCode,USGS_siteno,USGS_StationName,XcoordUTMNAD83_m,YcoordUTMNAD83_m,OnNorthWellTransect,DistanceFromCenterOfNorthOilBody_m,...,CasingMaterialDescription,ScreenMaterialDescription,DateOfConstruction,Comments_DateOfConstruction,NameOfContractor,DrillerName,Comments_Miscellaneous,SiteActiveStatus,Comments_Status,ApproxRemovalDate
206,x2101,Bemidji,ww,groundwater observation well,x473430095051401,2101 Bemidji Toxics Research Site,343039.96030,5271168.79500,NaN,NaN,...,NaN,NaN,20210621.00000,NaN,USGS MN Water Science Center,"A. Berg, Z. Hillman",Located near Unnamed Lake,Active,NaN,NaN
207,x2103A,Bemidji,ww,groundwater observation well,x473429095051402,2103A Bemidji Toxics Research Site,343067.19120,5271160.05300,NaN,NaN,...,NaN,NaN,20210622.00000,NaN,USGS MN Water Science Center,"A. Berg, Z. Hillman",Located near Unnamed Lake shoreline,Active,NaN,NaN
208,x2103B,Bemidji,ww,groundwater observation well,x473429095051403,2103B Bemidji Toxics Research Site,343067.74930,5271159.85500,NaN,NaN,...,NaN,NaN,20210623.00000,NaN,USGS MN Water Science Center,"A. Berg, Z. Hillman",Located near Unnamed Lake shoreline,Active,NaN,NaN
209,x2103C,Bemidji,ww,groundwater observation well,x473429095051404,2103C Bemidji Toxics Research Site,343066.52130,5271160.26200,NaN,NaN,...,NaN,NaN,20210623.00000,NaN,USGS MN Water Science Center,"A. Berg, Z. Hillman",Located near Unnamed Lake shoreline,Active,NaN,NaN
210,x2104A,Bemidji,ww,groundwater observation well,x473429095051405,2104A Bemidji Toxics Research Site,343044.36730,5271153.90300,NaN,NaN,...,NaN,NaN,20210623.00000,NaN,USGS MN Water Science Center,"A. Berg, Z. Hillman",Located on hillside near Unnamed Lake,Active,NaN,NaN
211,x2104B,Bemidji,ww,groundwater observation well,x473429095051406,2104B Bemidji Toxics Research Site,343044.19930,5271153.15200,NaN,NaN,...,NaN,NaN,20210624.00000,NaN,USGS MN Water Science Center,"A. Berg, Z. Hillman",Located on hillside near Unnamed Lake,Active,NaN,NaN
212,x2104C,Bemidji,ww,groundwater observation well,x473429095051407,2104C Bemidji Toxics Research Site,343044.50180,5271154.52800,NaN,NaN,...,NaN,NaN,20210625.00000,NaN,USGS MN Water Science Center,"A. Berg, Z. Hillman",Located on hillside near Unnamed Lake,Active,NaN,NaN
213,x2102F,Bemidji,wf,failed well installation (or any installation),NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,20210622.00000,"Attempted installation on 06/22/21, but well c...",USGS MN Water Science Center,"A. Berg, Z. Hillman",Located near 2105 well nest. This was a failed...,Removed,"failed well, no well installed",NaN
214,x2105A,Bemidji,ww,groundwater observation well,x473430095051501,2105A Bemidji Toxics Research Site 000018...,343011.59960,5271150.56900,NaN,NaN,...,NaN,NaN,20210815.00000,NaN,USGS MN Water Science Center,"B. Mason, J. Letsos","Located on top of hill, down-gradient of 955 w...",Active,NaN,NaN
215,x2105B,Bemidji,ww,groundwater observation well,x473430095051505,2105B Bemidji Toxics Research Site 000018...,343012.66810,5271151.24400,NaN,NaN,...,NaN,NaN,20210815.00000,NaN,USGS MN Water Science Center,"B. Mason, J. Letsos","Located on top of hill, down-gradient of 955 w...",Active,NaN,NaN


In [49]:
# directory
dir = r'P:\0083\analysis\DataCompilation\DataCompilationPy\create_site_info_files\data_outputs'
# today's date YYYYMMDD
date = '20250220'
# file 
file_path = f'{dir}\{date}_datarelease\DataRelease_WellConstructionInformation.csv'
# read into csv
chk = pd.read_csv(file_path, encoding='windows-1252')
chk[chk['LocalSiteName'].str.startswith('x21', na=False)]

<string>:6: SyntaxWarning: invalid escape sequence '\{'
<>:6: SyntaxWarning: invalid escape sequence '\{'
<>:6: SyntaxWarning: invalid escape sequence '\D'
<string>:6: SyntaxWarning: invalid escape sequence '\{'
<>:6: SyntaxWarning: invalid escape sequence '\{'
<>:6: SyntaxWarning: invalid escape sequence '\D'
C:\Users\bmilinic\AppData\Local\Temp\1\ipykernel_14040\72554406.py:6: SyntaxWarning: invalid escape sequence '\{'
  file_path = f'{dir}\{date}_datarelease\DataRelease_WellConstructionInformation.csv'
C:\Users\bmilinic\AppData\Local\Temp\1\ipykernel_14040\72554406.py:6: SyntaxWarning: invalid escape sequence '\D'
  file_path = f'{dir}\{date}_datarelease\DataRelease_WellConstructionInformation.csv'
<string>:6: SyntaxWarning: invalid escape sequence '\{'


,LocalSiteName,StudySite,LocalUseCode,Comments_UseCode,USGS_siteno,USGS_StationName,XcoordUTMNAD83_m,YcoordUTMNAD83_m,OnNorthWellTransect,DistanceFromCenterOfNorthOilBody_m,...,CasingMaterialDescription,ScreenMaterialDescription,DateOfConstruction,Comments_DateOfConstruction,NameOfContractor,DrillerName,Comments_Miscellaneous,SiteActiveStatus,Comments_Status,ApproxRemovalDate
206,x2101,Bemidji,ww,groundwater observation well,x473430095051401,2101 Bemidji Toxics Research Site,343039.96030,5271168.79500,NaN,NaN,...,Steel-Galvanized,Stainless Steel,20210621.00000,NaN,USGS MN Water Science Center,"A. Berg, Z. Hillman",Located near Unnamed Lake,Active,NaN,NaN
207,x2101,Bemidji,ww,groundwater observation well,x473430095051401,2101 Bemidji Toxics Research Site,343039.96030,5271168.79500,NaN,NaN,...,Stainless Steel,NaN,20210621.00000,NaN,USGS MN Water Science Center,"A. Berg, Z. Hillman",Located near Unnamed Lake,Active,NaN,NaN
208,x2103A,Bemidji,ww,groundwater observation well,x473429095051402,2103A Bemidji Toxics Research Site,343067.19120,5271160.05300,NaN,NaN,...,Steel-Galvanized,Stainless Steel,20210622.00000,NaN,USGS MN Water Science Center,"A. Berg, Z. Hillman",Located near Unnamed Lake shoreline,Active,NaN,NaN
209,x2103B,Bemidji,ww,groundwater observation well,x473429095051403,2103B Bemidji Toxics Research Site,343067.74930,5271159.85500,NaN,NaN,...,Steel-Galvanized,Stainless Steel,20210623.00000,NaN,USGS MN Water Science Center,"A. Berg, Z. Hillman",Located near Unnamed Lake shoreline,Active,NaN,NaN
210,x2103B,Bemidji,ww,groundwater observation well,x473429095051403,2103B Bemidji Toxics Research Site,343067.74930,5271159.85500,NaN,NaN,...,Stainless Steel,NaN,20210623.00000,NaN,USGS MN Water Science Center,"A. Berg, Z. Hillman",Located near Unnamed Lake shoreline,Active,NaN,NaN
211,x2103C,Bemidji,ww,groundwater observation well,x473429095051404,2103C Bemidji Toxics Research Site,343066.52130,5271160.26200,NaN,NaN,...,Steel-Galvanized,Stainless Steel,20210623.00000,NaN,USGS MN Water Science Center,"A. Berg, Z. Hillman",Located near Unnamed Lake shoreline,Active,NaN,NaN
212,x2103C,Bemidji,ww,groundwater observation well,x473429095051404,2103C Bemidji Toxics Research Site,343066.52130,5271160.26200,NaN,NaN,...,Stainless Steel,NaN,20210623.00000,NaN,USGS MN Water Science Center,"A. Berg, Z. Hillman",Located near Unnamed Lake shoreline,Active,NaN,NaN
213,x2104A,Bemidji,ww,groundwater observation well,x473429095051405,2104A Bemidji Toxics Research Site,343044.36730,5271153.90300,NaN,NaN,...,Steel-Galvanized,Stainless Steel,20210623.00000,NaN,USGS MN Water Science Center,"A. Berg, Z. Hillman",Located on hillside near Unnamed Lake,Active,NaN,NaN
214,x2104B,Bemidji,ww,groundwater observation well,x473429095051406,2104B Bemidji Toxics Research Site,343044.19930,5271153.15200,NaN,NaN,...,Steel-Galvanized,Stainless Steel,20210624.00000,NaN,USGS MN Water Science Center,"A. Berg, Z. Hillman",Located on hillside near Unnamed Lake,Active,NaN,NaN
215,x2104B,Bemidji,ww,groundwater observation well,x473429095051406,2104B Bemidji Toxics Research Site,343044.19930,5271153.15200,NaN,NaN,...,Stainless Steel,NaN,20210624.00000,NaN,USGS MN Water Science Center,"A. Berg, Z. Hillman",Located on hillside near Unnamed Lake,Active,NaN,NaN


# extra tests

In [78]:
# Define the list of site numbers extracted from the image
site_numbers = [
    "x473430095051401",
    "x473429095051406",
    "x473429095051403",
    "x473429095051404",
    "x473429095051407"
]

# Ensure the column is treated as a string and filter the DataFrame
chk_filtered = tblWells3[tblWells3['USGS_siteno'].astype(str).isin(site_numbers)]

# Display the filtered DataFrame
chk_filtered

,LocalSiteName,StudySite,LocalUseCode,Comments_UseCode,USGS_siteno,USGS_StationName,XcoordUTMNAD83_m,YcoordUTMNAD83_m,OnNorthWellTransect,DistanceFromCenterOfNorthOilBody_m,...,CasingMaterialDescription,ScreenMaterialDescription,DateOfConstruction,Comments_DateOfConstruction,NameOfContractor,DrillerName,Comments_Miscellaneous,SiteActiveStatus,Comments_Status,ApproxRemovalDate
988,x2101,Bemidji,ww,groundwater observation well,x473430095051401,2101 Bemidji Toxics Research Site,343039.9603,5271168.795,NaN,NaN,...,Steel-Galvanized,Stainless Steel,20210621.0,NaN,USGS MN Water Science Center,"A. Berg, Z. Hillman",Located near Unnamed Lake,Active,NaN,NaN
989,x2101,Bemidji,ww,groundwater observation well,x473430095051401,2101 Bemidji Toxics Research Site,343039.9603,5271168.795,NaN,NaN,...,Stainless Steel,NaN,20210621.0,NaN,USGS MN Water Science Center,"A. Berg, Z. Hillman",Located near Unnamed Lake,Active,NaN,NaN
991,x2103B,Bemidji,ww,groundwater observation well,x473429095051403,2103B Bemidji Toxics Research Site,343067.7493,5271159.855,NaN,NaN,...,Steel-Galvanized,Stainless Steel,20210623.0,NaN,USGS MN Water Science Center,"A. Berg, Z. Hillman",Located near Unnamed Lake shoreline,Active,NaN,NaN
992,x2103B,Bemidji,ww,groundwater observation well,x473429095051403,2103B Bemidji Toxics Research Site,343067.7493,5271159.855,NaN,NaN,...,Stainless Steel,NaN,20210623.0,NaN,USGS MN Water Science Center,"A. Berg, Z. Hillman",Located near Unnamed Lake shoreline,Active,NaN,NaN
993,x2103C,Bemidji,ww,groundwater observation well,x473429095051404,2103C Bemidji Toxics Research Site,343066.5213,5271160.262,NaN,NaN,...,Steel-Galvanized,Stainless Steel,20210623.0,NaN,USGS MN Water Science Center,"A. Berg, Z. Hillman",Located near Unnamed Lake shoreline,Active,NaN,NaN
994,x2103C,Bemidji,ww,groundwater observation well,x473429095051404,2103C Bemidji Toxics Research Site,343066.5213,5271160.262,NaN,NaN,...,Stainless Steel,NaN,20210623.0,NaN,USGS MN Water Science Center,"A. Berg, Z. Hillman",Located near Unnamed Lake shoreline,Active,NaN,NaN
996,x2104B,Bemidji,ww,groundwater observation well,x473429095051406,2104B Bemidji Toxics Research Site,343044.1993,5271153.152,NaN,NaN,...,Steel-Galvanized,Stainless Steel,20210624.0,NaN,USGS MN Water Science Center,"A. Berg, Z. Hillman",Located on hillside near Unnamed Lake,Active,NaN,NaN
997,x2104B,Bemidji,ww,groundwater observation well,x473429095051406,2104B Bemidji Toxics Research Site,343044.1993,5271153.152,NaN,NaN,...,Stainless Steel,NaN,20210624.0,NaN,USGS MN Water Science Center,"A. Berg, Z. Hillman",Located on hillside near Unnamed Lake,Active,NaN,NaN
998,x2104C,Bemidji,ww,groundwater observation well,x473429095051407,2104C Bemidji Toxics Research Site,343044.5018,5271154.528,NaN,NaN,...,Steel-Galvanized,Stainless Steel,20210625.0,NaN,USGS MN Water Science Center,"A. Berg, Z. Hillman",Located on hillside near Unnamed Lake,Active,NaN,NaN
999,x2104C,Bemidji,ww,groundwater observation well,x473429095051407,2104C Bemidji Toxics Research Site,343044.5018,5271154.528,NaN,NaN,...,Stainless Steel,NaN,20210625.0,NaN,USGS MN Water Science Center,"A. Berg, Z. Hillman",Located on hillside near Unnamed Lake,Active,NaN,NaN


In [ ]:
#check specific rows for specific columns
dfcomplete.loc[0:5, ['LocalSiteName','USGS_siteno','LandSurfaceElevation_mASL_NAVD88', 'MLR_alt_va']]

,LocalSiteName,USGS_siteno,LandSurfaceElevation_mASL_NAVD88,MLR_alt_va
2,15-N8,NaN,NaN,NaN
3,15-N9,NaN,NaN,NaN
1,15-N7,NaN,NaN,NaN
5,15-N11,NaN,NaN,NaN
0,15-N6,NaN,NaN,NaN


In [ ]:
#check random rows for specific columns
dfcomplete.loc[:, ['LocalSiteName','USGS_siteno','LandSurfaceElevation_mASL_NAVD88', 'MLR_alt_va']].sample(5)

,LocalSiteName,USGS_siteno,LandSurfaceElevation_mASL_NAVD88,MLR_alt_va
733,1104,NaN,432.728,NaN
1585,8417B,473425095052003.000,432.441,432.441
1618,9027,473428095052203.000,428.095,428.095
350,L310-1.5,NaN,433.453,NaN
54,17.lk1,NaN,NaN,NaN


In [ ]:
# look at rows where specfic columns have specific values
dfcomplete.loc[dfcomplete['GWSI_USGS_siteno'] == 473429095051006, ['GWSI_USGS_siteno','LandSurfaceElevation_mASL_NAVD88']]

,GWSI_USGS_siteno,LandSurfaceElevation_mASL_NAVD88
8,473429095051006.000,426.296


In [ ]:
dfrmk3

,GWSI_AgencyCode,GWSI_USGS_siteno,LocalSiteName,GWSI_GWSI_RMK,GWSI_GWSI_RMK_Date,GWSI_GWSI_RMK_SequenceNo
0,USGS,x473429095051006,x1217E,This well was re-surveyed on 27 June 2019 by J...,20190730.000,1.000
1,USGS,x473429095051006,x1217E,Digital levels were then ran from the temporar...,20190730.000,2.000
2,USGS,x473429095051006,x1217E,This well casing is constructed of stainless s...,20190730.000,3.000
3,USGS,x473424095052912,x1602G-01,"This is port ""01"" for vadose zone vapor/gas sa...",20200206.000,1.000
4,USGS,x473424095052906,x707G-01,"This is port ""01"" for vadose zone vapor/gas sa...",20200206.000,1.000
...,...,...,...,...,...,...
628,USGS,x473419095052503,xWG,This is the location of a wetland staff gage f...,20190730.000,1.000
629,USGS,x473419095052304,x1509G-01,"This is port ""01"" for vadose zone vapor/gas sa...",20200206.000,1.000
630,USGS,x473425095051601,x1707,This is a monitoring well used for the Bemidji...,20170726.000,1.000
631,USGS,x473423095051501,x1708,This monitoring well used for the Bemidji Toxi...,20170726.000,1.000


In [ ]:
# another example but with a string
dfcomplete.loc[dfcomplete['LocalSiteName'] == '15-N6', ['GWSI_USGS_siteno','LocalSiteName','LandSurfaceElevation_mASL_NAVD88']]

,GWSI_USGS_siteno,LocalSiteName,LandSurfaceElevation_mASL_NAVD88
0,NaN,15-N6,434.049


In [ ]:
# another example but with can extra conditional where each boolean needs round brackets
dfrmk3.loc[(dfrmk3['GWSI_USGS_siteno'] == 'x473429095051006') & (dfrmk3['GWSI_GWSI_RMK_SequenceNo'] == 1), :]

,GWSI_AgencyCode,GWSI_USGS_siteno,LocalSiteName,GWSI_GWSI_RMK,GWSI_GWSI_RMK_Date,GWSI_GWSI_RMK_SequenceNo
0,USGS,x473429095051006,x1217E,This well was re-surveyed on 27 June 2019 by J...,20190730.000,1.000


In [ ]:
#PRINT OUT DF KEYS IN ABC ORDER
dfcomplete.reindex(sorted(dfcomplete.columns), axis=1).keys()

Index(['AQ_AppliedByUser', 'AQ_AppliedTime', 'AQ_Comment',
       'AQ_DecommissionedDate', 'AQ_DecommissionedReason', 'AQ_Description',
       'AQ_Elevation', 'AQ_IsMeasuredAgainstLocalAssumedDatum',
       'AQ_MeasurementDirection', 'AQ_Method',
       ...
       'WELL_WellheadCasingLength', 'WELL_WellheadCasingProtection',
       'WELL_WellheadCompletion', 'WELL_WidthOfOpeningsInOpenInterval_inches',
       'WellCasingInnerDiameter_cm', 'WellSite',
       'WidthOfOpeningsInOpenInterval_cm', 'XcoordUTMNAD83_m',
       'YcoordUTMNAD83_m', 'well_MP_height_m'],
      dtype='object', length=380)

In [ ]:
# look at the number of unique values in a column
len(pd.unique(dfcomplete['LandSurfaceElevation_mASL_NAVD88'])) 

1158

In [ ]:
# number of columns and rows in a df
mastersitelist2.shape

(1729, 21)

# Comparing the old outputs with my updated outputs and these newest outputs

In [ ]:
print(os.listdir()) # creates a list of directories
print(os.scandir()) # creates an object which can be iterated through

['.ipynb_checkpoints', '01_inputfiles_prep-Copy1.ipynb', '01_inputfiles_prep.ipynb', '02_data_convergence-Copy1.ipynb', '02_data_convergence-Copy2-20221222.ipynb', '02_data_convergence-Copy3-20221228.ipynb', '02_data_convergence.ipynb', '02_data_convergenceCopy4-20230102.ipynb', 'ColumnMapping', 'config', 'datacheck', 'DataRelease_Well_nofilter.csv', 'data_inputs', 'data_outputs', 'description', 'HEAD', 'hooks', 'info', 'objects', 'refs', 'Untitled-1.ipynb', 'Untitled-1_20230102.ipynb']


In [ ]:
# import old output files from R 
path = r'data_outputs/old_fromR/'
dir = os.listdir(path)
oldc = pd.read_csv(path+dir[0], encoding='Windows-1252') # Coreinformation
oldm = pd.read_csv(path+dir[1], encoding='Windows-1252') # mastersitelist
oldo = pd.read_csv(path+dir[2], encoding='Windows-1252') # otherequipment
oldw = pd.read_csv(path+dir[3], encoding='Windows-1252') # wellconstruction
# import updated output files where R code was translated to Python code
path = r'data_outputs/20230110_datarelease/'
dir = os.listdir(path)
midc = pd.read_csv(path+dir[0])
midm = pd.read_csv(path+dir[1])
mido = pd.read_csv(path+dir[2])
midr = pd.read_csv(path+dir[3]) # rmk3
midw = pd.read_csv(path+dir[4])
# import (1/19/2023) output files from Python where everything was reorganized
path = r'data_outputs/20230119_datarelease/'
dir = os.listdir(path)
newc = pd.read_csv(path+dir[0])
newm = pd.read_csv(path+dir[1])
newo = pd.read_csv(path+dir[2])
newr = pd.read_csv(path+dir[3]) # rmk3
neww = pd.read_csv(path+dir[4])

In [ ]:
# .shape prints out the (rows, cols) of a dataframe. Cols should generally match. 
# Rows Could have slightly increased from old to mid since mid is slightly updated. 
# New files may have slightly less than mid as some duplicate rows were deleted
print("Core Info:", oldc.shape, midc.shape, newc.shape)
print("MasterSiteList Info:", oldm.shape, midm.shape, newm.shape)
print("OtherEquipment Info:", oldo.shape, mido.shape, newo.shape)
print("Well Con Info:", oldw.shape, midw.shape, neww.shape)
print("RMK Info:", 'nan', midr.shape, newr.shape)

Core Info: (1408, 30) (1470, 28) (1469, 28)
MasterSiteList Info: (1701, 21) (1731, 21) (1729, 21)
OtherEquipment Info: (434, 20) (433, 20) (423, 20)
Well Con Info: (605, 33) (615, 33) (596, 33)
RMK Info: nan (633, 6) (633, 6)


In [ ]:
# reorganized the same data
row_names = ["Core Info:", "MasterSiteList Info:","OtherEquipment Info:","Well Con Info:","RMK Info:"]
col_names = ["Old R dataframe", 'Translated to Py', 'Reorganized in Py']
data = [[oldc.shape, midc.shape, newc.shape],[oldm.shape, midm.shape, newm.shape],
        [oldo.shape, mido.shape, newo.shape], [oldw.shape, midw.shape, neww.shape],['nan', midr.shape, newr.shape]]
df = pd.DataFrame(data, columns=col_names)
df.index = [row_names]
df

,Old R dataframe,Translated to Py,Reorganized in Py
Core Info:,"(1408, 30)","(1470, 28)","(1469, 28)"
MasterSiteList Info:,"(1701, 21)","(1731, 21)","(1729, 21)"
OtherEquipment Info:,"(434, 20)","(433, 20)","(423, 20)"
Well Con Info:,"(605, 33)","(615, 33)","(596, 33)"
RMK Info:,nan,"(633, 6)","(633, 6)"


I notice that core info lost some columns

In [ ]:
# get the column labels of each dataframe
colsOLD = set(oldc.columns)
colsNEW = set(newc.columns)

# find the columns missing in df2 compared to df1
missing_in_NEW = colsOLD.difference(colsNEW)
print("Cols missing the new dataframe:", missing_in_NEW)

# find the columns missing in df1 compared to df2
missing_in_OLD = colsNEW.difference(colsOLD)
print("Cols missing from the old dataframe:", missing_in_OLD)


Cols missing the new dataframe: {'Unnamed: 28', 'Unnamed: 29', 'TotalBoring_DrillingDepth_mBLS'}
Cols missing from the old dataframe: {'TotalBoring/DrillingDepth_mBLS'}


In [ ]:
missing_cols = colsOLD.symmetric_difference(colsNEW)
print(missing_cols)
missing_cols = colsNEW.symmetric_difference(colsOLD)
print(missing_cols)

{'TotalBoring/DrillingDepth_mBLS', 'Unnamed: 28', 'Unnamed: 29', 'TotalBoring_DrillingDepth_mBLS'}
{'Unnamed: 28', 'TotalBoring/DrillingDepth_mBLS', 'Unnamed: 29', 'TotalBoring_DrillingDepth_mBLS'}


Turns out there are just invisible columns in the old dataframe called "Unnamed: ##". Perhaps it has to do with the weirdness of the encoding type. Can be ignored and know that both dataframes actally have 28 rows for Coring dataframes

### All columns match up...what about data?

In [ ]:
print(oldm['LocalSiteName'].dtypes)
print(newm['LocalSiteName'].dtypes)
# First I have to make the old columns match
oldm['LocalSiteName'] = 'x' + oldm['LocalSiteName'].astype(str)

object
object


In [ ]:
display(oldm.loc[0:2, :])
display(newm.loc[0:2, :])

,LocalSiteName,StudySite,AgencyCode,USGS_siteno,USGS_StationName,XcoordUTMNAD83_m,YcoordUTMNAD83_m,LandSurfaceElevation_mASL_NAVD88,OnNorthWellTransect,DistanceFromCenterOfNorthOilBody_m,...,CoreSite,OtherEquipmentSite,TopOfScreenElevation_mASL_NAVD88,BottomOfScreenElevation_mASL_NAVD88,MidOfScreenElevation_mASL_NAVD88,DateOfConstruction,Comments_DateOfConstruction,SiteActiveStatus,Comments_Status,ApproxRemovalDate
0,x15-N6,Bemidji,USGS,NaN,NaN,342541.280,5271062.290,434.049,NaN,NaN,...,1,0,NaN,NaN,NaN,NaN,NaN,Active,NaN,NaN
1,x15-N7,Bemidji,USGS,NaN,NaN,342563.440,5271111.220,434.401,NaN,NaN,...,1,0,NaN,NaN,NaN,NaN,NaN,Active,NaN,NaN
2,x15-N8,Bemidji,USGS,NaN,NaN,342557.790,5271104.470,434.263,NaN,NaN,...,1,0,NaN,NaN,NaN,NaN,NaN,Active,NaN,NaN


,LocalSiteName,StudySite,AgencyCode,USGS_siteno,USGS_StationName,XcoordUTMNAD83_m,YcoordUTMNAD83_m,LandSurfaceElevation_mASL_NAVD88,OnNorthWellTransect,DistanceFromCenterOfNorthOilBody_m,...,CoreSite,OtherEquipmentSite,TopOfScreenElevation_mASL_NAVD88,BottomOfScreenElevation_mASL_NAVD88,MidOfScreenElevation_mASL_NAVD88,DateOfConstruction,Comments_DateOfConstruction,SiteActiveStatus,Comments_Status,ApproxRemovalDate
0,x15-N6,Bemidji,USGS,NaN,NaN,342541.280,5271062.290,434.049,NaN,NaN,...,1,0,NaN,NaN,NaN,NaN,NaN,Active,NaN,NaN
1,x15-N7,Bemidji,USGS,NaN,NaN,342563.440,5271111.220,434.401,NaN,NaN,...,1,0,NaN,NaN,NaN,NaN,NaN,Active,NaN,NaN
2,x15-N8,Bemidji,USGS,NaN,NaN,342557.790,5271104.470,434.263,NaN,NaN,...,1,0,NaN,NaN,NaN,NaN,NaN,Active,NaN,NaN


In [ ]:
# check true versus false cols
check = oldm['LocalSiteName'].isin(newm['LocalSiteName'])
true_count = check.sum()
false_count = len(check) - true_count
print("Number of True values:", true_count)
print("Number of False values:", false_count)

Number of True values: 1591
Number of False values: 110


Lesson on boolean indexing...a boolean (true/false) mask can be used to select rows where elements showed up to be true or false. Typically it would return true values

> true_rows=oldm.loc[check]

But it can also return false values by using the '~' operator which negates the mask 

> false_rows=oldm.loc[~check]

Another way to do it is

> false_rows=oldm.loc[check == False]

In [ ]:
false_rows=oldm.loc[~check]
false_rows.sample(10)

,LocalSiteName,StudySite,AgencyCode,USGS_siteno,USGS_StationName,XcoordUTMNAD83_m,YcoordUTMNAD83_m,LandSurfaceElevation_mASL_NAVD88,OnNorthWellTransect,DistanceFromCenterOfNorthOilBody_m,...,CoreSite,OtherEquipmentSite,TopOfScreenElevation_mASL_NAVD88,BottomOfScreenElevation_mASL_NAVD88,MidOfScreenElevation_mASL_NAVD88,DateOfConstruction,Comments_DateOfConstruction,SiteActiveStatus,Comments_Status,ApproxRemovalDate
374,x325,Bemidji,USGS,NaN,NaN,342821.940,5271050.470,432.800,1.000,37.200,...,1,0,NaN,NaN,NaN,20030725.000,NaN,Active,NaN,NaN
528,x804,Bemidji,USGS,NaN,NaN,342773.300,5271035.370,430.212,1.000,-13.390,...,1,0,NaN,NaN,NaN,20080719.000,NaN,Active,NaN,NaN
1604,xTelephone bo 3,Bemidji,USGS,NaN,NaN,342755.710,5271025.740,430.811,1.000,-33.370,...,0,1,NaN,NaN,NaN,NaN,Installation date unknown,Active,NaN,NaN
341,x133,Bemidji,USGS,NaN,NaN,342792.660,5271024.320,430.741,NaN,NaN,...,1,0,NaN,NaN,NaN,20010726.000,NaN,Active,NaN,NaN
318,x36,Bemidji,USGS,NaN,NaN,342804.980,5270859.210,425.626,NaN,NaN,...,1,0,NaN,NaN,NaN,20000808.000,NaN,Active,NaN,NaN
311,x27,Bemidji,USGS,NaN,NaN,342622.060,5270942.200,433.209,1.000,-189.410,...,1,0,NaN,NaN,NaN,20000805.000,NaN,Active,NaN,NaN
527,x803,Bemidji,USGS,NaN,NaN,342899.400,5271090.880,433.381,1.000,124.390,...,1,0,NaN,NaN,NaN,20080723.000,NaN,Active,NaN,NaN
504,x601,Bemidji,USGS,NaN,NaN,342846.270,5271062.810,433.041,1.000,64.440,...,1,0,NaN,NaN,NaN,20060718.000,NaN,Active,NaN,NaN
337,x129,Bemidji,USGS,NaN,NaN,342791.430,5271034.010,430.557,NaN,NaN,...,1,0,NaN,NaN,NaN,20010726.000,NaN,Active,NaN,NaN
379,x330,Bemidji,USGS,NaN,NaN,342861.940,5271069.470,433.207,1.000,81.470,...,1,0,NaN,NaN,NaN,20030726.000,NaN,Active,NaN,NaN


In [ ]:
display(oldm.loc[0:2, :])
display(newm.loc[0:2, :])

Why are there over 100 False values for LocalSiteNames? There seems to be small ignorable problems. Like the error in the old dataframe where 0007 was converted to 7. Or "Telephone box" versus "telephone bo". 

## I can repeat this process using any dataframe or column

In [ ]:
display(oldo.LocalSiteName)
display(newo.LocalSiteName)

0              BM1
1      CL-pipeline
2           F1212G
3           F1302G
4           WT1601
          ...     
429        15.p100
430         15.q93
431       old.tdr1
432       old.tdr2
433       old.tdr3
Name: LocalSiteName, Length: 434, dtype: object

0           xWT1601
1      xCL-pipeline
2        xL1802-1.8
3        xL1802-3.3
4          xF530G.1
           ...     
418         xWT1003
419         xWT1004
420         xWT1201
421         xWT1301
422         xWT1712
Name: LocalSiteName, Length: 423, dtype: object

In [ ]:
# manually do any processing on the old dfs to match the new dfs format
oldo['LocalSiteName'] = 'x' + oldo['LocalSiteName'].astype(str)

In [ ]:
#inputs
old_df = oldo # do these values appear in... 
new_df = newo # this dataframe?
col_name = 'LocalSiteName'
# check true versus false cols
check = old_df[col_name].isin(new_df[col_name])
true_count = check.sum()
false_count = len(check) - true_count
print("Number of True values:", true_count)
print("Number of False values:", false_count)

false_rows=old_df.loc[~check]
display(false_rows[['LocalSiteName', 'Comments']])

Number of True values: 419
Number of False values: 15


,LocalSiteName,Comments
19,xB-pipeline,record added to database to accommodate pipeli...
76,xTelephone bo 3,NaN
77,xPower bo 6,NaN
78,xPower bo 6 meter,NaN
79,xPower bo 7,NaN
80,xTelephone bo 4,NaN
81,xTelephone bo 5,NaN
102,xPower bo 1,NaN
103,xPower bo 2,NaN
104,xPower bo 3,NaN


In [ ]:
# how to display the whole text in the column
pd.options.display.max_colwidth = 100
display(false_rows[['LocalSiteName', 'Comments']])
pd.reset_option('max_colwidth') # turn output display back to default to avoid crashing the computer if printing a lot of columns

In [ ]:
#checked rows where mismatches occur

display(old_df.loc[old_df['LocalSiteName'] == '17.lk.sc5', :])
display(new_df.loc[new_df['LocalSiteName'] == 'x17.lk.sc5', :])

,LocalSiteName,StudySite,LocalUseCode,Comments_UseCode,USGS_siteno,USGS_StationName,XcoordUTMNAD83_m,YcoordUTMNAD83_m,OnNorthWellTransect,DistanceFromCenterOfNorthOilBody_m,...,CasingMaterialDescription,ScreenMaterialDescription,DateOfConstruction,Comments_DateOfConstruction,NameOfContractor,DrillerName,Comments_Miscellaneous,SiteActiveStatus,Comments_Status,ApproxRemovalDate
19,17.lk.sc5,Bemidji,NaN,NaN,NaN,NaN,343127.640,5271096.410,NaN,NaN,...,NaN,NaN,NaN,NaN,USGS MN Water Science Center,Not applicable,NaN,Removed,temporary sampling location,NaN


,LocalSiteName,StudySite,LocalUseCode,Comments_UseCode,USGS_siteno,USGS_StationName,XcoordUTMNAD83_m,YcoordUTMNAD83_m,OnNorthWellTransect,DistanceFromCenterOfNorthOilBody_m,...,CasingMaterialDescription,ScreenMaterialDescription,DateOfConstruction,Comments_DateOfConstruction,NameOfContractor,DrillerName,Comments_Miscellaneous,SiteActiveStatus,Comments_Status,ApproxRemovalDate


All values that are in **'XcoordUTMNAD83_m'** in the old df also appear in the new df for mastersitelist, core, and otherequipment but not for Well Construction. Are rows with '17.lk.sc5' really needed? 

## Convert to UTF-8 example. 
Since the encoding type "Windows-1252" is limited with its character use (single byte!) and can't be translated to all languages, it is good practice in general to convert it to UTF-8 so it can handle the wide variety of characters in different languages like Chinese. 

In [ ]:
#detect the encoding of a file
import chardet
with open(r'data_outputs/old_fromR/DataRelease01_CoreInformation.csv', 'rb') as f:
    result = chardet.detect(f.read())
    print(result)

{'encoding': 'Windows-1252', 'confidence': 0.73, 'language': ''}


In [ ]:

oldc.to_csv(r'data_outputs/old_fromR/Test_DataRelease01_CoreInformation_UTF8.csv', index=False, encoding='UTF-8')


In [ ]:
test = pd.read_csv(r'data_outputs/old_fromR/DataRelease01_CoreInformation_UTF8.csv')

## Testing to see if any data from Aquarius (reference points) has instances where one site has multiple valid MP heights for the same day

In [ ]:
path = r'data_inputs/aquarius'
dir = os.listdir(path)
aq = pd.read_csv(path+'/Referencepoints_fromPy.csv')#, encoding='Windows-1252')

In [ ]:
aq.sort_values(by='site', ascending=True)

,UniqueId,Name,Description,site,IsMeasuredAgainstLocalAssumedDatum,ValidFrom,Unit,Elevation,Uncertainty,Method,MeasurementDirection,Comment,AppliedTime,AppliedByUser,DecommissionedDate,DecommissionedReason
0,0868303bbc6c441982fff320c2d0a44f,MP-1 height,"No MP exists, but 0.00 ft. was populated for f...",473356095043701,True,0001-01-01T00:00:00.0000000+00:00,ft,0.000,0.050,GNSS3,FromTopToBottom,"No MP exists, but 0.00 ft. was populated for f...",2021-03-02T13:12:52.2229750-06:00,aberg,NaN,NaN
1,00ddcc72af874a04a11e2735b1524b39,MP-1 height,"MP assumed to be the top of casing, not protec...",473358095061401,True,1984-10-17T00:00:00.0000000-06:00,ft,3.000,NaN,NaN,FromTopToBottom,From GW_MPNT 1,2021-01-12T20:13:22.9820002-06:00,admin,NaN,NaN
2,b570bd5035cb4d0db9f55cce351f4fc6,MP-1 height,"MP assumed to be the top of casing, not protec...",473404095054101,True,1984-06-19T00:00:00.0000000-06:00,ft,2.630,NaN,NaN,FromTopToBottom,From GW_MPNT 1,2021-01-12T20:13:25.7468950-06:00,admin,NaN,NaN
3,b6a787ea9c1d4625b02dacca2b76a2c6,MP-1 height,"MP assumed to be the top of casing, not protec...",473405095060101,True,1984-05-08T00:00:00.0000000-06:00,ft,1.410,NaN,NaN,FromTopToBottom,From GW_MPNT 1,2021-01-12T20:13:26.3601610-06:00,admin,NaN,NaN
4,4d6975e06cda4e32a5df7224776d21e2,MP-1 height,"MP assumed to be the top of casing, not protec...",473405095060201,True,1985-07-09T00:00:00.0000000-06:00,ft,2.710,NaN,NaN,FromTopToBottom,From GW_MPNT 1,2021-01-12T20:13:26.6997530-06:00,admin,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
568,6c0293e1fd884a97a49cdc79c66961c2,MP-1 height,MP prior to well being bent due to logging,473437095052401,True,1985-07-08T00:00:00.0000000-06:00,ft,2.350,NaN,NaN,FromTopToBottom,From GW_MPNT 1,2021-01-12T20:15:37.0271760-06:00,admin,2004-12-01T00:00:00.0000000-06:00,GW MP ended on 2004-11-30
569,ec11dddc0a49444db3b113b7c5270c1a,Generic Zero Height,Generic Zero Height Reference Point,473437095052401,True,0001-01-01T00:00:00.0000000+00:00,ft,0.000,NaN,NaN,FromTopToBottom,NaN,2021-01-12T20:15:37.1758930-06:00,admin,NaN,NaN
570,f534bdda01e1488d9a9f17eb94ff3db1,MP-1 height,"No MP exists, but 0.00 ft. was populated for f...",473439095063001,True,0001-01-01T00:00:00.0000000+00:00,ft,0.000,0.090,GNSS2,FromTopToBottom,"No MP exists, but 0.00 ft. was populated for f...",2021-03-02T10:41:27.7681060-06:00,aberg,NaN,NaN
571,4e4ffdaf38264739b398c38fc7effae9,MP-1 height,"MP assumed to be the top of casing, not protec...",473440095063001,True,1984-10-17T00:00:00.0000000-06:00,ft,1.880,NaN,NaN,FromTopToBottom,From GW_MPNT 1,2021-01-12T20:15:38.3859010-06:00,admin,NaN,NaN


In [ ]:
duplicates = aq[aq.duplicated(subset='ValidFrom', keep=False) & aq.duplicated(subset='site', keep=False)]
duplicates

,UniqueId,Name,Description,site,IsMeasuredAgainstLocalAssumedDatum,ValidFrom,Unit,Elevation,Uncertainty,Method,MeasurementDirection,Comment,AppliedTime,AppliedByUser,DecommissionedDate,DecommissionedReason
6,454ebe23596f4309b0d4bfd5af7afc5f,Generic Zero Height,Generic Zero Height Reference Point,473411095062901,True,0001-01-01T00:00:00.0000000+00:00,ft,0.000,NaN,NaN,FromTopToBottom,NaN,2021-01-12T20:13:28.1128920-06:00,admin,NaN,NaN
7,2dfd9a7470a641ceb700e208c2a54e01,MP-1 height,"MP assumed to be the top of casing, not protec...",473411095062901,True,1984-06-19T00:00:00.0000000-06:00,ft,2.940,NaN,NaN,FromTopToBottom,From GW_MPNT 1,2021-01-12T20:13:28.0593842-06:00,admin,NaN,NaN
9,c20a2cc81c3c4897b7a707e0ec4f142e,Generic Zero Height,Generic Zero Height Reference Point,473413095053701,True,0001-01-01T00:00:00.0000000+00:00,ft,0.000,NaN,NaN,FromTopToBottom,NaN,2021-01-12T20:13:29.5128960-06:00,admin,NaN,NaN
10,fb7a18b7001949f2b996131625ab52e6,MP-2 height,More accurate MP length. This should be used f...,473413095053701,True,1985-05-16T00:00:00.0000000-06:00,ft,2.560,NaN,NaN,FromTopToBottom,From GW_MPNT 2,2021-01-12T20:13:29.4114640-06:00,admin,NaN,NaN
11,8ea8bbed63164e71b25a9d410bf420d1,MP-1 height,Initial MP for this well. Discovered to be ina...,473413095053701,True,1985-05-15T00:00:00.0000000-06:00,ft,2.690,NaN,NaN,FromTopToBottom,From GW_MPNT 1,2021-01-12T20:13:29.3543652-06:00,admin,1985-05-16T00:00:00.0000000-06:00,GW MP ended on 1985-05-15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
557,593d69f35c3e46b796c26172cb5f617b,Generic Zero Height,Generic Zero Height Reference Point,473430095052201,True,0001-01-01T00:00:00.0000000+00:00,ft,0.000,NaN,NaN,FromTopToBottom,NaN,2021-01-12T20:15:31.4800930-06:00,admin,NaN,NaN
560,a197c4beca9f488db5bb6b61b1aaed7e,MP-1 height,"MP assumed to be the top of casing, not protec...",473431095052601,True,1983-05-25T00:00:00.0000000-06:00,ft,1.460,NaN,NaN,FromTopToBottom,From GW_MPNT 1,2021-01-12T20:15:33.1546890-06:00,admin,NaN,NaN
561,dc03d54712a24dc0bc6c3e4c2818ee55,Generic Zero Height,Generic Zero Height Reference Point,473431095052601,True,0001-01-01T00:00:00.0000000+00:00,ft,0.000,NaN,NaN,FromTopToBottom,NaN,2021-01-12T20:15:33.2391440-06:00,admin,NaN,NaN
567,40592487f5434867a231b0ce478e62b2,MP-2 height,New MP after well had been bent from logging,473437095052401,True,2004-12-01T00:00:00.0000000-06:00,ft,2.110,NaN,NaN,FromTopToBottom,From GW_MPNT 2,2021-01-12T20:15:37.0842790-06:00,admin,NaN,NaN


In [ ]:
duplicates = aq[aq.duplicated(subset=['ValidFrom', 'site'], keep=False)]
duplicates

,UniqueId,Name,Description,site,IsMeasuredAgainstLocalAssumedDatum,ValidFrom,Unit,Elevation,Uncertainty,Method,MeasurementDirection,Comment,AppliedTime,AppliedByUser,DecommissionedDate,DecommissionedReason
71,e0c6350eb279430e93c1d6b07172d6e5,MP-1 height,"MP assumed to be the top of casing, not protec...",473420095052406,True,1983-06-01T00:00:00.0000000-06:00,ft,1.840,NaN,NaN,FromTopToBottom,From GW_MPNT 1,2021-01-12T20:13:46.9615040-06:00,admin,1983-06-02T00:00:00.0000000-06:00,GW MP ended on 1983-06-01
72,9c91b87395aa407a8aba7bf4829169f9,MP-2 height,More accurate MP length. This should be used f...,473420095052406,True,1983-06-01T00:00:00.0000000-06:00,ft,1.220,NaN,NaN,FromTopToBottom,From GW_MPNT 2,2021-01-12T20:13:47.0120730-06:00,admin,NaN,NaN


In [ ]:
aq.sort_values(by='site', ascending=True, inplace=True)
dups = aq[aq.duplicated(subset='ValidFrom', keep=False)]

                             UniqueId                 Name  \
0    0868303bbc6c441982fff320c2d0a44f          MP-1 height   
1    00ddcc72af874a04a11e2735b1524b39          MP-1 height   
2    b570bd5035cb4d0db9f55cce351f4fc6          MP-1 height   
3    b6a787ea9c1d4625b02dacca2b76a2c6          MP-1 height   
5    21199dd5b3cc431fa8e382391281708b          MP-1 height   
..                                ...                  ...   
567  40592487f5434867a231b0ce478e62b2          MP-2 height   
569  ec11dddc0a49444db3b113b7c5270c1a  Generic Zero Height   
570  f534bdda01e1488d9a9f17eb94ff3db1          MP-1 height   
571  4e4ffdaf38264739b398c38fc7effae9          MP-1 height   
572  01134a800425416097a51eb47c8c92bd          MP-1 height   

                                           Description             site  \
0    No MP exists, but 0.00 ft. was populated for f...  473356095043701   
1    MP assumed to be the top of casing, not protec...  473358095061401   
2    MP assumed to be the top 

In [ ]:
dups_both = aq[aq.duplicated(subset='site', keep=False)]

In [ ]:
n = 25
dups_both.iloc[n:n+10]

,UniqueId,Name,Description,site,IsMeasuredAgainstLocalAssumedDatum,ValidFrom,Unit,Elevation,Uncertainty,Method,MeasurementDirection,Comment,AppliedTime,AppliedByUser,DecommissionedDate,DecommissionedReason
57,78cb527bdfb34510a3b3e1e45c5c5554,Generic Zero Height,Generic Zero Height Reference Point,473419095052701,True,0001-01-01T00:00:00.0000000+00:00,ft,0.000,NaN,NaN,FromTopToBottom,NaN,2021-01-12T20:13:43.1946160-06:00,admin,NaN,NaN
58,40c5447dd34a4178a0547703dd2aec9b,MP-1 height,Initial MP for this well. Discovered to be ina...,473419095052701,True,1990-06-24T00:00:00.0000000-06:00,ft,2.330,NaN,NaN,FromTopToBottom,From GW_MPNT 1,2021-01-12T20:13:43.0588542-06:00,admin,1990-06-25T00:00:00.0000000-06:00,GW MP ended on 1990-06-24
61,b189ac0c44484f88aeb479290e19c82a,Generic Zero Height,Generic Zero Height Reference Point,473420095051601,True,0001-01-01T00:00:00.0000000+00:00,ft,0.000,NaN,NaN,FromTopToBottom,NaN,2021-01-12T20:13:44.6435950-06:00,admin,NaN,NaN
62,6242d9c744994208a836cbf3506d8afc,MP-2 height,Measured stickup on 03/09/10,473420095051601,True,2010-03-09T00:00:00.0000000-06:00,ft,3.020,NaN,NaN,FromTopToBottom,From GW_MPNT 2,2021-01-12T20:13:44.5718442-06:00,admin,NaN,NaN
63,fe75a9fe9a0f4fbe95eae0e7b314266c,MP-1 height,Initial stickup after unknown installation date.,473420095051601,True,1983-05-24T00:00:00.0000000-06:00,ft,3.080,NaN,NaN,FromTopToBottom,From GW_MPNT 1,2021-01-12T20:13:44.5093000-06:00,admin,2010-03-09T00:00:00.0000000-06:00,GW MP ended on 2010-03-08
67,9f27134526ec4eafa19a0a69ed5ca827,Generic Zero Height,Generic Zero Height Reference Point,473420095052401,True,0001-01-01T00:00:00.0000000+00:00,ft,0.000,NaN,NaN,FromTopToBottom,NaN,2021-01-12T20:13:45.8577620-06:00,admin,NaN,NaN
66,8968582fcc824744b56d9a5a8058a8e6,MP-1 height,"MP assumed to be the top of casing, not protec...",473420095052401,True,1983-05-25T00:00:00.0000000-06:00,ft,1.930,NaN,NaN,FromTopToBottom,From GW_MPNT 1,2021-01-12T20:13:45.8006598-06:00,admin,NaN,NaN
69,aa5efbb38fa943fbbeb661d0b5ce203c,MP-2 height,More accurate MP length. This should be used f...,473420095052403,True,1989-10-27T00:00:00.0000000-06:00,ft,1.310,NaN,NaN,FromTopToBottom,From GW_MPNT 2,2021-01-12T20:13:46.2121028-06:00,admin,NaN,NaN
70,dc0196e348d44b51bbb6b6880728f452,Generic Zero Height,Generic Zero Height Reference Point,473420095052403,True,0001-01-01T00:00:00.0000000+00:00,ft,0.000,NaN,NaN,FromTopToBottom,NaN,2021-01-12T20:13:46.2779150-06:00,admin,NaN,NaN
68,3c1844c9ace64827bcf4eb933844ab3a,MP-1 height,Initial MP for this well. Discovered to be ina...,473420095052403,True,1989-10-26T00:00:00.0000000-06:00,ft,1.600,NaN,NaN,FromTopToBottom,From GW_MPNT 1,2021-01-12T20:13:46.1558040-06:00,admin,1989-10-27T00:00:00.0000000-06:00,GW MP ended on 1989-10-26


In [ ]:
display(aq.loc[aq['LocalSiteName'] == 'x17.lk.sc5', :])
aq.loc

,UniqueId,Name,Description,site,IsMeasuredAgainstLocalAssumedDatum,ValidFrom,Unit,Elevation,Uncertainty,Method,MeasurementDirection,Comment,AppliedTime,AppliedByUser,DecommissionedDate,DecommissionedReason
0,0868303bbc6c441982fff320c2d0a44f,MP-1 height,"No MP exists, but 0.00 ft. was populated for f...",473356095043701,True,0001-01-01T00:00:00.0000000+00:00,ft,0.000,0.050,GNSS3,FromTopToBottom,"No MP exists, but 0.00 ft. was populated for f...",2021-03-02T13:12:52.2229750-06:00,aberg,NaN,NaN
1,00ddcc72af874a04a11e2735b1524b39,MP-1 height,"MP assumed to be the top of casing, not protec...",473358095061401,True,1984-10-17T00:00:00.0000000-06:00,ft,3.000,NaN,NaN,FromTopToBottom,From GW_MPNT 1,2021-01-12T20:13:22.9820002-06:00,admin,NaN,NaN
2,b570bd5035cb4d0db9f55cce351f4fc6,MP-1 height,"MP assumed to be the top of casing, not protec...",473404095054101,True,1984-06-19T00:00:00.0000000-06:00,ft,2.630,NaN,NaN,FromTopToBottom,From GW_MPNT 1,2021-01-12T20:13:25.7468950-06:00,admin,NaN,NaN
3,b6a787ea9c1d4625b02dacca2b76a2c6,MP-1 height,"MP assumed to be the top of casing, not protec...",473405095060101,True,1984-05-08T00:00:00.0000000-06:00,ft,1.410,NaN,NaN,FromTopToBottom,From GW_MPNT 1,2021-01-12T20:13:26.3601610-06:00,admin,NaN,NaN
4,4d6975e06cda4e32a5df7224776d21e2,MP-1 height,"MP assumed to be the top of casing, not protec...",473405095060201,True,1985-07-09T00:00:00.0000000-06:00,ft,2.710,NaN,NaN,FromTopToBottom,From GW_MPNT 1,2021-01-12T20:13:26.6997530-06:00,admin,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
568,6c0293e1fd884a97a49cdc79c66961c2,MP-1 height,MP prior to well being bent due to logging,473437095052401,True,1985-07-08T00:00:00.0000000-06:00,ft,2.350,NaN,NaN,FromTopToBottom,From GW_MPNT 1,2021-01-12T20:15:37.0271760-06:00,admin,2004-12-01T00:00:00.0000000-06:00,GW MP ended on 2004-11-30
569,ec11dddc0a49444db3b113b7c5270c1a,Generic Zero Height,Generic Zero Height Reference Point,473437095052401,True,0001-01-01T00:00:00.0000000+00:00,ft,0.000,NaN,NaN,FromTopToBottom,NaN,2021-01-12T20:15:37.1758930-06:00,admin,NaN,NaN
570,f534bdda01e1488d9a9f17eb94ff3db1,MP-1 height,"No MP exists, but 0.00 ft. was populated for f...",473439095063001,True,0001-01-01T00:00:00.0000000+00:00,ft,0.000,0.090,GNSS2,FromTopToBottom,"No MP exists, but 0.00 ft. was populated for f...",2021-03-02T10:41:27.7681060-06:00,aberg,NaN,NaN
571,4e4ffdaf38264739b398c38fc7effae9,MP-1 height,"MP assumed to be the top of casing, not protec...",473440095063001,True,1984-10-17T00:00:00.0000000-06:00,ft,1.880,NaN,NaN,FromTopToBottom,From GW_MPNT 1,2021-01-12T20:15:38.3859010-06:00,admin,NaN,NaN


## Another comparison

In [3]:
# read in
fpath = r"P:\0083\analysis\DataCompilation\DataCompilationPy\create_site_info_files\data_outputs\20230523_datarelease\DataRelease_MasterSiteList.csv"
dfpy = pd.read_csv(fpath)

fpath = r"P:\0083\analysis\DataCompilation\DataCompilationPy\create_site_info_files\data_outputs\20240703_datarelease\DataRelease_MasterSiteList.csv"
dfpynew = pd.read_csv(fpath)

fpath = r"P:\0083\analysis\DataCompilation\DataCompilationPy\create_site_info_files\data_outputs\old_fromR\DataRelease01_MasterSiteList.csv"
dfr = pd.read_csv(fpath)

In [4]:
# add column
dfpy['origin'] = 'python'
dfpynew['origin'] = 'python_updated'
dfr['origin'] = 'R'

# update r column to match python
dfr['LocalSiteName'] = 'x' + dfr['LocalSiteName'].astype(str)

In [9]:
dfcompare = pd.concat([dfpy, dfpynew, dfr], axis=0)
dfcompare = pd.concat([dfpynew, dfr], axis=0)

In [10]:
# move origin col to 0th position

# pop out the origin col
col_origin = dfcompare.pop('origin')
# insert it at the 0th pos
dfcompare.insert(0, 'origin', col_origin)

In [13]:
def pop_pop(df, col_name):
    col = df.pop(col_name)
    df.insert(0, col_name, col)

pop_pop(dfcompare, 'DistanceFromCenterOfNorthOilBody_m')

In [11]:
dfcompare.sort_values(by=['LocalSiteName', 'XcoordUTMNAD83_m'], inplace=True)

In [ ]:
dfcompare.loc[:,['origin', 'LocalSiteName','XcoordUTMNAD83_m', 'LandSurfaceElevation_mASL_NAVD88', ]]

In [8]:
dfcompare.head(2)

,origin,LocalSiteName,StudySite,AgencyCode,USGS_siteno,USGS_StationName,XcoordUTMNAD83_m,YcoordUTMNAD83_m,LandSurfaceElevation_mASL_NAVD88,OnNorthWellTransect,...,CoreSite,OtherEquipmentSite,TopOfScreenElevation_mASL_NAVD88,BottomOfScreenElevation_mASL_NAVD88,MidOfScreenElevation_mASL_NAVD88,DateOfConstruction,Comments_DateOfConstruction,SiteActiveStatus,Comments_Status,ApproxRemovalDate
294,python,x0001-0004,Bemidji,USGS,NaN,NaN,342793.57,5271049.07,432.918,1.0,...,1,0,NaN,NaN,NaN,20000731.0,NaN,Active,NaN,NaN
294,python_updated,x0001-0004,Bemidji,USGS,NaN,NaN,342793.57,5271049.07,432.918,1.0,...,1,0,NaN,NaN,NaN,20000731.0,NaN,Active,NaN,NaN


### SHIFT

In [53]:
display(dfcompare['origin'])
display(dfcompare['origin'].shift(-1))

294                 R
294            python
294    python_updated
295                 R
295            python
            ...      
208            python
208    python_updated
209                 R
209            python
209    python_updated
Name: origin, Length: 5159, dtype: object

294            python
294    python_updated
294                 R
295            python
295    python_updated
            ...      
208    python_updated
208                 R
209            python
209    python_updated
209               NaN
Name: origin, Length: 5159, dtype: object

In [59]:
dfcompare['pattern'] = dfcompare['origin'].shift(1) + dfcompare['origin'] + dfcompare['origin'].shift(-1)

In [60]:
dfcompare['pattern']

294                      NaN
294    Rpythonpython_updated
294    pythonpython_updatedR
295    python_updatedRpython
295    Rpythonpython_updated
               ...          
208    Rpythonpython_updated
208    pythonpython_updatedR
209    python_updatedRpython
209    Rpythonpython_updated
209                      NaN
Name: pattern, Length: 5159, dtype: object

In [63]:
df = dfcompare[dfcompare['pattern'].str.contains('RR', na=False)]

In [64]:
df

,origin,LocalSiteName,StudySite,AgencyCode,USGS_siteno,USGS_StationName,XcoordUTMNAD83_m,YcoordUTMNAD83_m,LandSurfaceElevation_mASL_NAVD88,OnNorthWellTransect,...,OtherEquipmentSite,TopOfScreenElevation_mASL_NAVD88,BottomOfScreenElevation_mASL_NAVD88,MidOfScreenElevation_mASL_NAVD88,DateOfConstruction,Comments_DateOfConstruction,SiteActiveStatus,Comments_Status,ApproxRemovalDate,pattern
319,R,x101,Bemidji,USGS,NaN,NaN,342830.010,5271053.560,432.853,1.000,...,0,NaN,NaN,NaN,20010718.000,NaN,Active,NaN,NaN,python_updatedRR
552,R,x1010-16,Bemidji,USGS,NaN,NaN,342805.570,5271047.180,432.720,1.000,...,1,NaN,NaN,NaN,20100721.000,NaN,Active,NaN,NaN,RRpython
320,R,x102,Bemidji,USGS,NaN,NaN,342804.830,5271041.830,432.312,1.000,...,0,NaN,NaN,NaN,20010718.000,NaN,Active,NaN,NaN,python_updatedRR
554,R,x1026-38,Bemidji,USGS,NaN,NaN,342807.510,5271046.630,432.550,1.000,...,0,NaN,NaN,NaN,20100724.000,NaN,Active,NaN,NaN,RRpython
321,R,x103,Bemidji,USGS,NaN,NaN,342763.060,5271033.220,430.131,1.000,...,0,NaN,NaN,NaN,20010718.000,NaN,Active,NaN,NaN,python_updatedRR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1603,R,xTelephone bo 2,Bemidji,USGS,NaN,NaN,342806.450,5271065.300,433.998,NaN,...,1,NaN,NaN,NaN,NaN,Installation date unknown,Active,NaN,NaN,RRR
1604,R,xTelephone bo 3,Bemidji,USGS,NaN,NaN,342755.710,5271025.740,430.811,1.000,...,1,NaN,NaN,NaN,NaN,Installation date unknown,Active,NaN,NaN,RRR
1605,R,xTelephone bo 4,Bemidji,USGS,NaN,NaN,342854.410,5270935.910,428.121,NaN,...,1,NaN,NaN,NaN,NaN,Installation date unknown,Active,NaN,NaN,RRR
1606,R,xTelephone bo 5,Bemidji,USGS,NaN,NaN,342673.990,5270986.770,431.240,1.000,...,1,NaN,NaN,NaN,NaN,Installation date unknown,Active,NaN,NaN,RRR


In [65]:
df = dfcompare
display(df.loc[df['LocalSiteName'] == 'x121', :])

,origin,LocalSiteName,StudySite,AgencyCode,USGS_siteno,USGS_StationName,XcoordUTMNAD83_m,YcoordUTMNAD83_m,LandSurfaceElevation_mASL_NAVD88,OnNorthWellTransect,...,OtherEquipmentSite,TopOfScreenElevation_mASL_NAVD88,BottomOfScreenElevation_mASL_NAVD88,MidOfScreenElevation_mASL_NAVD88,DateOfConstruction,Comments_DateOfConstruction,SiteActiveStatus,Comments_Status,ApproxRemovalDate,pattern
330,R,x121,Bemidji,USGS,NaN,NaN,342849.130,5271060.420,432.947,1.000,...,0,NaN,NaN,NaN,20010724.000,NaN,Active,NaN,NaN,python_updatedRR


0 = mastersitelist2  
1 = mastersitelist  
2 = dfcomplete  
 
---

## Check if col exists and if value in col exists

In [95]:
df = dfmlr
col_name = 'site_tp_cd'

if col_name in df.columns:
    print('columns exists...')
else:
    print('ERROR: column does not exist')

display(df.loc[df[col_name] == 'x121', :].shape)
display(df.loc[df[col_name] == '121', :].shape)
display(df.loc[df[col_name] == 121, :].shape)

columns exists...


(0, 42)

(0, 42)

(0, 42)